In [16]:
"""
===============================================
    TENDER WEBSCRAPING - NO LOGIN PLATFORMS
    BAK Economics - Constructor Academy
===============================================

Scraping public tender platforms (no authentication):
✅ TED API (EU Tenders) - Official API
✅ BSV (Swiss Research) - Public website
✅ COR Europa (EU Committee) - Public website

Author: Cedric & Team
Date: September 2025
"""

# Install required packages
!pip install requests beautifulsoup4 pandas

print("Setup complete!")
print("Packages: requests, beautifulsoup4, pandas")


Setup complete!
Packages: requests, beautifulsoup4, pandas


In [4]:
"""
Install Selenium
"""

!pip install selenium

print("✅ Selenium installed!")
print("\nNote: You also need ChromeDriver")
print("Install with: brew install chromedriver")
print("Or download from: https://chromedriver.chromium.org/")

✅ Selenium installed!

Note: You also need ChromeDriver
Install with: brew install chromedriver
Or download from: https://chromedriver.chromium.org/


In [17]:
# Install both packages in your notebook
!pip install selenium webdriver-manager

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import json

# Pandas display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 80)


In [7]:
"""
BAK ECONOMICS SEARCH CRITERIA
==============================
Based on company's actual methodology from their documentation:

"When we look for tenders in economic analysis and research on platforms 
like Simap, we search for buzzwords like 'BFS', 'Bundesamt für Statistik', 
or within vocabulary section '72000000 Consulting, market and economic research'"
"""

# Key buzzwords BAK Economics uses
BAK_KEYWORDS = [
    'BFS',                          # Bundesamt für Statistik (Federal Statistical Office)
    'Bundesamt für Statistik',
    'Bundesamt',
    'economic research',
    'Wirtschaftsforschung',
    'economic analysis',
    'Wirtschaftsanalyse',
    'market research',
    'Marktforschung',
    'statistics',
    'Statistik',
    'survey',
    'Umfrage',
    'Konjunkturforschung',
    'economic consulting'
]

# CPV Codes (Common Procurement Vocabulary) relevant to BAK
BAK_CPV_CODES = {
    '72000000': 'IT services: consulting, software development, Internet and support',
    '73000000': 'Research and experimental development services',
    '73100000': 'Research and experimental development services',
    '73110000': 'Research services',
    '79300000': 'Market and economic research; polling and statistics',
    '79310000': 'Market research services',
    '79311000': 'Survey services',
    '79320000': 'Public-opinion polling services',
    '79330000': 'Statistical services',
    '79400000': 'Business and management consultancy services',
    '79410000': 'Business consultancy services',
    '79411000': 'General management consultancy services'
}

print("✅ BAK Economics search criteria loaded!")
print(f"📋 {len(BAK_KEYWORDS)} keywords configured")
print(f"🔢 {len(BAK_CPV_CODES)} CPV codes configured")
print(f"\nTop keywords: {', '.join(BAK_KEYWORDS[:5])}")

✅ BAK Economics search criteria loaded!
📋 15 keywords configured
🔢 12 CPV codes configured

Top keywords: BFS, Bundesamt für Statistik, Bundesamt, economic research, Wirtschaftsforschung


In [ ]:
"""
TED FIX - METHOD 1: Bulk XML Download (not needed)
======================================
Official TED data distribution method
Most reliable and complete
"""

def download_ted_bulk_xml():
    """
    Access TED bulk XML downloads
    This is the official way to get TED data
    """
    
    print("📦 TED BULK XML DOWNLOAD METHOD")
    print("="*70)
    
    # TED bulk download page
    bulk_url = "https://data.europa.eu/data/datasets/ted-csv?locale=en"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)'
    }
    
    try:
        response = requests.get(bulk_url, headers=headers, timeout=15)
        
        if response.status_code == 200:
            print("✅ TED bulk download page accessible\n")
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Find download links
            links = soup.find_all('a', href=True)
            download_links = []
            
            for link in links:
                href = link['href']
                text = link.text.strip()
                
                if any(word in href.lower() for word in ['.csv', '.xml', 'download', 'export']):
                    download_links.append({
                        'Text': text[:60],
                        'URL': href
                    })
            
            print("📋 Available download options:")
            if download_links:
                for i, dl in enumerate(download_links[:5], 1):
                    print(f"  {i}. {dl['Text']}")
            
            print("\n💡 HOW TO USE:")
            print("  1. Visit: https://data.europa.eu/data/datasets/ted-csv")
            print("  2. Download CSV/XML file (daily or monthly)")
            print("  3. Load into pandas:")
            print("     df = pd.read_csv('ted_export.csv')")
            print("  4. Filter by BAK keywords")
            
            print("\n📊 DATA STRUCTURE:")
            print("  • Contract notices")
            print("  • Award notices")
            print("  • CPV codes")
            print("  • Full tender descriptions")
            
            return True
        else:
            print(f"❌ Failed: {response.status_code}")
            return False
            
    except Exception as e:
        print(f"❌ Error: {e}")
        return False

# TEST
download_ted_bulk_xml()

In [ ]:
"""
EU PORTAL SCRAPER WITH KEYWORD MATCHING
"""

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time

# BAK Keywords
BAK_INCLUSION = [
    'study', 'analysis', 'economy', 'economic', 'benchmarking', 
    'market research', 'statistics', 'survey', 'index', 'economic research',
    'Studie', 'Analyse', 'Wirtschaft', 'Ökonomie', 'Wirtschaftsberatung',
    'Marktforschung', 'Wirtschaftsforschung', 'Umfragen', 'Statistiken',
    'Bundesamt für Statistik', 'BKS', 'SECO', 'Bundesamt für'
]

BAK_EXCLUSION = [
    'construction', 'health care', 'transport', 'mobility', 'sport', 
    'culture', 'street', 'infrastructure', 'IT', 'Bau', 'Gesundheit', 
    'Verkehr', 'Mobilität', 'Kultur', 'Strasse', 'Infrastruktur', 'Prozess'
]

def scrape_eu_portal():
    
    print("EU PORTAL SCRAPER")
    print("="*70)
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    
    try:
        # Login
        driver.get("https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/home")
        time.sleep(3)
        
        try:
            driver.find_element(By.LINK_TEXT, "Login").click()
            time.sleep(2)
            driver.find_element(By.ID, "username").send_keys("jansencedric37@gmail.com")
            driver.find_element(By.ID, "password").send_keys("Bully1245??")
            driver.find_element(By.NAME, "submit").click()
            time.sleep(5)
        except:
            pass
        
        # Load tenders
        url = "https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/calls-for-tenders?order=DESC&pageNumber=1&pageSize=50&sortBy=startDate&isExactMatch=true&status=31094502"
        
        driver.get(url)
        time.sleep(8)
        
        elements = driver.find_elements(By.CSS_SELECTOR, "div.ng-star-inserted")
        
        tenders = []
        for elem in elements:
            text = elem.text.strip()
            if text and len(text) > 30:
                try:
                    link = elem.find_element(By.TAG_NAME, "a").get_attribute("href")
                except:
                    link = "N/A"
                
                tenders.append({
                    'Platform': 'EU Portal',
                    'Content': text[:150],
                    'Full_Text': text,
                    'URL': link
                })
        
        driver.quit()
        df = pd.DataFrame(tenders)
        
        print(f"Scraped: {len(df)} tenders")
        
        if df.empty:
            return df
        
        # Filter
        inclusion_pattern = '|'.join([rf'\b{k}\b' for k in BAK_INCLUSION])
        has_inclusion = df['Full_Text'].str.contains(inclusion_pattern, case=False, na=False, regex=True)
        
        exclusion_pattern = '|'.join([rf'\b{k}\b' for k in BAK_EXCLUSION])
        has_exclusion = df['Full_Text'].str.contains(exclusion_pattern, case=False, na=False, regex=True)
        
        filtered = df[has_inclusion & ~has_exclusion]
        
        # Add matched keywords
        matched = []
        for idx, row in filtered.iterrows():
            text = row['Full_Text'].lower()
            matches = [k for k in BAK_INCLUSION if k.lower() in text]
            matched.append(', '.join(matches[:5]) if matches else 'None')
        
        filtered['Matched_Keywords'] = matched
        
        print(f"BAK-relevant: {len(filtered)}\n")
        
        # Display and save
        display(filtered[['Platform', 'Content', 'Matched_Keywords', 'URL']])
        
        filtered[['Platform', 'Content', 'Matched_Keywords', 'URL']].to_csv('eu_portal_results.csv', index=False)
        print("Saved: eu_portal_results.csv")
        
        return filtered
        
    except Exception as e:
        print(f"Error: {e}")
        driver.quit()
        return pd.DataFrame()

# Run
eu_data = scrape_eu_portal()

In [ ]:
"""
EU Portal - Clean Structured Data  (Draft)
"""

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time

BAK_INCLUSION = [
    'study', 'analysis', 'economy', 'economic', 'benchmarking', 
    'market research', 'statistics', 'survey', 'index', 'economic research',
    'Studie', 'Analyse', 'Wirtschaft', 'Marktforschung', 'Wirtschaftsforschung',
    'Bundesamt für Statistik', 'SECO', 'Bundesamt für'
]

BAK_EXCLUSION = [
    'construction', 'health care', 'transport', 'mobility', 'sport', 
    'culture', 'street', 'infrastructure',
    'Bau', 'Gesundheit', 'Verkehr', 'Mobilität', 'Kultur', 'Strasse', 'Infrastruktur'
]

def extract_tender_fields(driver, url):
    """Extract key tender fields"""
    try:
        driver.get(url)
        time.sleep(5)
        
        text = driver.find_element(By.TAG_NAME, "body").text
        lines = [l.strip() for l in text.split('\n') if l.strip()]
        
        fields = {
            'Procedure_ID': '',
            'Description': '',
            'Procedure_Type': '',
            'Budget': '',
            'Lead_Authority': '',
            'CPV_Code': '',
            'Contract_Duration': '',
            'Deadline': ''
        }
        
        for i, line in enumerate(lines):
            if i+1 >= len(lines):
                continue
            next_line = lines[i+1]
            
            if line == 'Procedure identifier':
                fields['Procedure_ID'] = next_line
            elif line == 'Description' and len(next_line) > 50:
                fields['Description'] = next_line[:250]
            elif line == 'Procedure type':
                fields['Procedure_Type'] = next_line
            elif line == 'Estimated total value':
                fields['Budget'] = next_line
            elif line == 'Lead contracting authority':
                fields['Lead_Authority'] = next_line
            elif 'Main classification (CPV)' in line:
                fields['CPV_Code'] = next_line
            elif line == 'Maximum contract duration':
                fields['Contract_Duration'] = next_line
            elif 'Deadline for receipt of tenders' in line:
                fields['Deadline'] = next_line
        
        return fields
    except:
        return None

def scrape_eu_open_tenders():
    
    print("EU PORTAL - OPEN TENDERS")
    print("="*70 + "\n")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    
    try:
        # Login
        driver.get("https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/home")
        time.sleep(3)
        try:
            driver.find_element(By.LINK_TEXT, "Login").click()
            time.sleep(2)
            driver.find_element(By.ID, "username").send_keys("jansencedric37@gmail.com")
            driver.find_element(By.ID, "password").send_keys("Bully1245??")
            driver.find_element(By.NAME, "submit").click()
            time.sleep(5)
        except:
            pass
        
        # Get open tenders
        url = "https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/calls-for-tenders?order=DESC&pageNumber=1&pageSize=50&sortBy=startDate&isExactMatch=true&status=31094502"
        driver.get(url)
        time.sleep(8)
        
        # Collect tenders
        elements = driver.find_elements(By.CSS_SELECTOR, "div.ng-star-inserted")
        all_tenders = []
        
        for elem in elements:
            text = elem.text.strip()
            if text and len(text) > 30:
                try:
                    link = elem.find_element(By.TAG_NAME, "a").get_attribute("href")
                    if link and link.startswith('http'):
                        all_tenders.append({'Title': text[:80], 'Full_Text': text, 'URL': link})
                except:
                    continue
        
        print(f"Total tenders: {len(all_tenders)}")
        
        # BAK filtering
        df = pd.DataFrame(all_tenders)
        inclusion_pattern = '|'.join([rf'\b{k}\b' for k in BAK_INCLUSION])
        has_inclusion = df['Full_Text'].str.contains(inclusion_pattern, case=False, na=False, regex=True)
        
        exclusion_pattern = '|'.join([rf'\b{k}\b' for k in BAK_EXCLUSION])
        has_exclusion = df['Full_Text'].str.contains(exclusion_pattern, case=False, na=False, regex=True)
        
        relevant = df[has_inclusion & ~has_exclusion]
        print(f"BAK-relevant: {len(relevant)}\n")
        
        # Extract details
        results = []
        for idx, row in relevant.iterrows():
            print(f"Extracting {len(results)+1}/{len(relevant)}...")
            
            fields = extract_tender_fields(driver, row['URL'])
            if fields:
                results.append({
                    'Title': row['Title'],
                    'Procedure_ID': fields['Procedure_ID'],
                    'Budget': fields['Budget'],
                    'Procedure_Type': fields['Procedure_Type'],
                    'Deadline': fields['Deadline'],
                    'Contract_Duration': fields['Contract_Duration'],
                    'Lead_Authority': fields['Lead_Authority'],
                    'CPV_Code': fields['CPV_Code'],
                    'Description': fields['Description'],
                    'URL': row['URL']
                })
            
            time.sleep(2)
        
        driver.quit()
        
        final_df = pd.DataFrame(results)
        
        print(f"\n{'='*70}")
        print(f"COMPLETE: {len(final_df)} open BAK-relevant tenders")
        print("="*70 + "\n")
        
        if not final_df.empty:
            display(final_df[['Title', 'Budget', 'Deadline', 'Procedure_Type']])
            final_df.to_csv('eu_open_tenders.csv', index=False)
            print("\nSaved: eu_open_tenders.csv")
        
        return final_df
        
    except Exception as e:
        print(f"Error: {e}")
        driver.quit()
        return pd.DataFrame()

# Run
eu_open = scrape_eu_open_tenders()

SyntaxError: invalid syntax (3744195664.py, line 2)

In [ ]:
"""
EU Portal - ALL Open Tenders with Full Details (Final Version)
"""

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time

BAK_INCLUSION = [
    'study', 'analysis', 'economy', 'economic', 'benchmarking', 
    'market research', 'statistics', 'survey', 'index', 'economic research',
    'Studie', 'Analyse', 'Wirtschaft', 'Marktforschung', 'Wirtschaftsforschung',
    'Bundesamt für Statistik', 'SECO', 'Bundesamt für'
]

BAK_EXCLUSION = [
    'construction', 'health care', 'transport', 'mobility', 'sport', 
    'culture', 'street', 'infrastructure',
    'Bau', 'Gesundheit', 'Verkehr', 'Mobilität', 'Kultur', 'Strasse', 'Infrastruktur'
]

def extract_tender_fields(driver, url):
    """Extract full tender details"""
    try:
        driver.get(url)
        time.sleep(5)
        
        text = driver.find_element(By.TAG_NAME, "body").text
        lines = [l.strip() for l in text.split('\n') if l.strip()]
        
        fields = {
            'Procedure_ID': '',
            'Description': '',
            'Procedure_Type': '',
            'Budget': '',
            'Lead_Authority': '',
            'CPV_Code': '',
            'Contract_Duration': '',
            'Deadline': ''
        }
        
        for i, line in enumerate(lines):
            if i+1 >= len(lines):
                continue
            next_line = lines[i+1]
            
            if line == 'Procedure identifier':
                fields['Procedure_ID'] = next_line
            elif line == 'Description' and len(next_line) > 50:
                fields['Description'] = next_line[:250]
            elif line == 'Procedure type':
                fields['Procedure_Type'] = next_line
            elif line == 'Estimated total value':
                fields['Budget'] = next_line
            elif line == 'Lead contracting authority':
                fields['Lead_Authority'] = next_line
            elif 'Main classification (CPV)' in line:
                fields['CPV_Code'] = next_line
            elif line == 'Maximum contract duration':
                fields['Contract_Duration'] = next_line
            elif 'Deadline for receipt of tenders' in line:
                fields['Deadline'] = next_line
        
        return fields
    except:
        return None

def scrape_all_open_tenders():
    
    print("EU PORTAL - ALL OPEN TENDERS")
    print("="*70 + "\n")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    
    try:
        # Login
        driver.get("https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/home")
        time.sleep(3)
        try:
            driver.find_element(By.LINK_TEXT, "Login").click()
            time.sleep(2)
            driver.find_element(By.ID, "username").send_keys("jansencedric37@gmail.com")
            driver.find_element(By.ID, "password").send_keys("Bully1245??")
            driver.find_element(By.NAME, "submit").click()
            time.sleep(5)
        except:
            pass
        
        # Scrape ALL pages
        base_url = "https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/calls-for-tenders?order=DESC&pageNumber={}&pageSize=50&sortBy=startDate&isExactMatch=true&status=31094502"
        
        all_tenders = []
        page = 1
        
        print("Step 1: Collecting all open tenders...")
        while True:
            url = base_url.format(page)
            driver.get(url)
            time.sleep(8)
            
            elements = driver.find_elements(By.CSS_SELECTOR, "div.ng-star-inserted")
            
            page_tenders = []
            for elem in elements:
                text = elem.text.strip()
                if text and len(text) > 30:
                    try:
                        link = elem.find_element(By.TAG_NAME, "a").get_attribute("href")
                        if link and link.startswith('http'):
                            page_tenders.append({'Title': text[:80], 'Full_Text': text, 'URL': link})
                    except:
                        continue
            
            if not page_tenders:
                print(f"  Page {page}: No more results\n")
                break
            
            all_tenders.extend(page_tenders)
            print(f"  Page {page}: {len(page_tenders)} tenders (Total: {len(all_tenders)})")
            
            page += 1
            time.sleep(3)
        
        print(f"Total open tenders scraped: {len(all_tenders)}")
        
        # Remove duplicates
        df = pd.DataFrame(all_tenders)
        df = df.drop_duplicates(subset='URL', keep='first')
        print(f"After deduplication: {len(df)} unique tenders\n")
        
        # BAK filtering
        print("Step 2: Applying BAK filtering...")
        inclusion_pattern = '|'.join([rf'\b{k}\b' for k in BAK_INCLUSION])
        has_inclusion = df['Full_Text'].str.contains(inclusion_pattern, case=False, na=False, regex=True)
        
        exclusion_pattern = '|'.join([rf'\b{k}\b' for k in BAK_EXCLUSION])
        has_exclusion = df['Full_Text'].str.contains(exclusion_pattern, case=False, na=False, regex=True)
        
        relevant = df[has_inclusion & ~has_exclusion]
        print(f"BAK-relevant open tenders: {len(relevant)}\n")
        
        if relevant.empty:
            print("No BAK-relevant tenders found")
            driver.quit()
            return pd.DataFrame()
        
        # Extract full details
        print("Step 3: Extracting full details...\n")
        results = []
        for idx, row in relevant.iterrows():
            print(f"  {len(results)+1}/{len(relevant)}: Extracting...")
            
            fields = extract_tender_fields(driver, row['URL'])
            if fields:
                results.append({
                    'Title': row['Title'],
                    'Procedure_ID': fields['Procedure_ID'],
                    'Budget': fields['Budget'],
                    'Procedure_Type': fields['Procedure_Type'],
                    'Deadline': fields['Deadline'],
                    'Contract_Duration': fields['Contract_Duration'],
                    'Lead_Authority': fields['Lead_Authority'],
                    'CPV_Code': fields['CPV_Code'],
                    'Description': fields['Description'],
                    'URL': row['URL']
                })
            
            time.sleep(2)
        
        driver.quit()
        
        final_df = pd.DataFrame(results)
        
        print(f"\n{'='*70}")
        print(f"COMPLETE: {len(final_df)} open BAK-relevant tenders")
        print("="*70 + "\n")
        
        if not final_df.empty:
            display(final_df[['Title', 'Budget', 'Deadline', 'Procedure_Type']])
            final_df.to_csv('eu_all_open_tenders.csv', index=False)
            print("\nSaved: eu_all_open_tenders.csv")
        
        return final_df
        
    except Exception as e:
        print(f"Error: {e}")
        driver.quit()
        return pd.DataFrame()

# Run
all_open = scrape_all_open_tenders()

In [ ]:
"""
EU Portal - Title Search Only (Simple Version)
"""

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time

def scrape_by_title():
    
    print("EU PORTAL - TITLE SEARCH ONLY")
    print("="*70)
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    
    try:
        # Login
        driver.get("https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/home")
        time.sleep(3)
        try:
            driver.find_element(By.LINK_TEXT, "Login").click()
            time.sleep(2)
            driver.find_element(By.ID, "username").send_keys("jansencedric37@gmail.com")
            driver.find_element(By.ID, "password").send_keys("Bully1245??")
            driver.find_element(By.NAME, "submit").click()
            time.sleep(5)
        except:
            pass
        
        # Get page 1 only for quick test
        url = "https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/calls-for-tenders?order=DESC&pageNumber=1&pageSize=50&sortBy=startDate&isExactMatch=true&status=31094502"
        driver.get(url)
        time.sleep(8)
        
        # Collect tenders
        elements = driver.find_elements(By.CSS_SELECTOR, "div.ng-star-inserted")
        tenders = []
        
        for elem in elements:
            text = elem.text.strip()
            if text and len(text) > 30:
                try:
                    link = elem.find_element(By.TAG_NAME, "a").get_attribute("href")
                    if link and link.startswith('http'):
                        # Extract title (first line only)
                        title = text.split('\n')[0]
                        tenders.append({'Title': title, 'URL': link})
                except:
                    continue
        
        driver.quit()
        
        df = pd.DataFrame(tenders).drop_duplicates(subset='Title')
        
        # Filter by title keywords
        keywords = ['economic', 'research', 'study', 'analysis', 'market', 'statistics']
        pattern = '|'.join(keywords)
        
        title_matches = df[df['Title'].str.contains(pattern, case=False, na=False)]
        
        print(f"\nTotal tenders: {len(df)}")
        print(f"Title matches: {len(title_matches)}\n")
        
        display(title_matches)
        title_matches.to_csv('eu_title_search.csv', index=False)
        print("Saved: eu_title_search.csv")
        
        return title_matches
        
    except Exception as e:
        print(f"Error: {e}")
        driver.quit()
        return pd.DataFrame()

# Run
title_results = scrape_by_title()

In [ ]:
"""
1. OPEN TENDERS - TITLE SEARCH - FULL BAK KEYWORDS
"""
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/home")
time.sleep(3)

try:
    driver.find_element(By.LINK_TEXT, "Login").click()
    time.sleep(2)
    driver.find_element(By.ID, "username").send_keys("jansencedric37@gmail.com")
    driver.find_element(By.ID, "password").send_keys("Bully1245??")
    driver.find_element(By.NAME, "submit").click()
    time.sleep(5)
except:
    pass

tenders = []
page = 1
while True:
    driver.get(f"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/calls-for-tenders?pageNumber={page}&status=31094502")
    time.sleep(8)
    elems = driver.find_elements(By.CSS_SELECTOR, "div.ng-star-inserted")
    found = []
    for e in elems:
        txt = e.text.strip()
        if txt and len(txt) > 30:
            try:
                link = e.find_element(By.TAG_NAME, "a").get_attribute("href")
                if link and link.startswith('http'):
                    found.append({'Title': txt.split('\n')[0], 'URL': link})
            except: 
                pass
    if not found: 
        break
    tenders.extend(found)
    print(f"Page {page}: {len(found)}")
    page += 1
    time.sleep(3)

driver.quit()

df = pd.DataFrame(tenders).drop_duplicates(subset='Title')

# Full BAK methodology
inclusion = ['study', 'Studie', 'analysis', 'Analyse', 'economy', 'Wirtschaft', 'Ökonomie', 
             'economic', 'benchmarking', 'Wirtschaftsberatung', 'market research', 'Marktforschung',
             'statistics', 'Statistiken', 'survey', 'Umfragen', 'index', 'Bundesamt für Statistik',
             'BKS', 'SECO', 'Staatssekretariat für Wirtschaft', 'Kanton Zürich', 'Kanton Luzern',
             'Bundesamt für', 'Regionen', 'Wirtschaftsforschung', 'Dienstleistung', 'Ausschreibung']

exclusion = ['construction', 'Bau', 'health care', 'Gesundheit', 'transport', 'Verkehr',
             'mobility', 'Mobilität', 'sport', 'culture', 'Kultur', 'street', 'Strasse',
             'infrastructure', 'Infrastruktur', 'process', 'Prozess']

inc_pattern = '|'.join([rf'\b{k}\b' for k in inclusion])
exc_pattern = '|'.join([rf'\b{k}\b' for k in exclusion])

has_inc = df['Title'].str.contains(inc_pattern, case=False, na=False, regex=True)
has_exc = df['Title'].str.contains(exc_pattern, case=False, na=False, regex=True)
matches = df[has_inc & ~has_exc]

matches.to_csv('open_title_full.csv', index=False)
print(f"\nOpen title (BAK filtered): {len(matches)}/{len(df)}")
display(matches)

In [ ]:
"""
2. CLOSED TENDERS - TITLE SEARCH - FULL BAK KEYWORDS
"""
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/home")
time.sleep(3)

try:
    driver.find_element(By.LINK_TEXT, "Login").click()
    time.sleep(2)
    driver.find_element(By.ID, "username").send_keys("jansencedric37@gmail.com")
    driver.find_element(By.ID, "password").send_keys("Bully1245??")
    driver.find_element(By.NAME, "submit").click()
    time.sleep(5)
except:
    pass

tenders = []
page = 1
while page <= 20:
    driver.get(f"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/calls-for-tenders?pageNumber={page}&status=31094501")
    time.sleep(8)
    elems = driver.find_elements(By.CSS_SELECTOR, "div.ng-star-inserted")
    found = []
    for e in elems:
        txt = e.text.strip()
        if txt and len(txt) > 30:
            try:
                link = e.find_element(By.TAG_NAME, "a").get_attribute("href")
                if link and link.startswith('http'):
                    found.append({'Title': txt.split('\n')[0], 'URL': link})
            except: 
                pass
    if not found: 
        break
    tenders.extend(found)
    print(f"Page {page}: {len(found)}")
    page += 1
    time.sleep(3)

driver.quit()

df = pd.DataFrame(tenders).drop_duplicates(subset='Title')

inclusion = ['study', 'Studie', 'analysis', 'Analyse', 'economy', 'Wirtschaft', 'Ökonomie', 
             'economic', 'benchmarking', 'Wirtschaftsberatung', 'market research', 'Marktforschung',
             'statistics', 'Statistiken', 'survey', 'Umfragen', 'index', 'Bundesamt für Statistik',
             'BKS', 'SECO', 'Staatssekretariat für Wirtschaft', 'Kanton Zürich', 'Kanton Luzern',
             'Bundesamt für', 'Regionen', 'Wirtschaftsforschung', 'Dienstleistung', 'Ausschreibung']

exclusion = ['construction', 'Bau', 'health care', 'Gesundheit', 'transport', 'Verkehr',
             'mobility', 'Mobilität', 'sport', 'culture', 'Kultur', 'street', 'Strasse',
             'infrastructure', 'Infrastruktur', 'process', 'Prozess']

inc_pattern = '|'.join([rf'\b{k}\b' for k in inclusion])
exc_pattern = '|'.join([rf'\b{k}\b' for k in exclusion])

has_inc = df['Title'].str.contains(inc_pattern, case=False, na=False, regex=True)
has_exc = df['Title'].str.contains(exc_pattern, case=False, na=False, regex=True)
matches = df[has_inc & ~has_exc]

matches.to_csv('closed_title_full.csv', index=False)
print(f"\nClosed title (BAK filtered): {len(matches)}/{len(df)}")
display(matches)

In [ ]:
"""
3. OPEN TENDERS - FULL TEXT - FULL BAK KEYWORDS
"""
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/home")
time.sleep(3)

try:
    driver.find_element(By.LINK_TEXT, "Login").click()
    time.sleep(2)
    driver.find_element(By.ID, "username").send_keys("jansencedric37@gmail.com")
    driver.find_element(By.ID, "password").send_keys("Bully1245??")
    driver.find_element(By.NAME, "submit").click()
    time.sleep(5)
except:
    pass

tenders = []
page = 1
while True:
    driver.get(f"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/calls-for-tenders?pageNumber={page}&status=31094502")
    time.sleep(8)
    elems = driver.find_elements(By.CSS_SELECTOR, "div.ng-star-inserted")
    found = []
    for e in elems:
        txt = e.text.strip()
        if txt and len(txt) > 30:
            try:
                link = e.find_element(By.TAG_NAME, "a").get_attribute("href")
                if link and link.startswith('http'):
                    found.append({'Title': txt.split('\n')[0], 'Full_Text': txt, 'URL': link})
            except: 
                pass
    if not found: 
        break
    tenders.extend(found)
    print(f"Page {page}: {len(found)}")
    page += 1
    time.sleep(3)

driver.quit()

df = pd.DataFrame(tenders).drop_duplicates(subset='URL')

inclusion = ['study', 'Studie', 'analysis', 'Analyse', 'economy', 'Wirtschaft', 'Ökonomie', 
             'economic', 'benchmarking', 'Wirtschaftsberatung', 'market research', 'Marktforschung',
             'statistics', 'Statistiken', 'survey', 'Umfragen', 'index', 'Bundesamt für Statistik',
             'BKS', 'SECO', 'Staatssekretariat für Wirtschaft', 'Kanton Zürich', 'Kanton Luzern',
             'Bundesamt für', 'Regionen', 'Wirtschaftsforschung', 'Dienstleistung', 'Ausschreibung']

exclusion = ['construction', 'Bau', 'health care', 'Gesundheit', 'transport', 'Verkehr',
             'mobility', 'Mobilität', 'sport', 'culture', 'Kultur', 'street', 'Strasse',
             'infrastructure', 'Infrastruktur', 'process', 'Prozess']

inc_pattern = '|'.join([rf'\b{k}\b' for k in inclusion])
exc_pattern = '|'.join([rf'\b{k}\b' for k in exclusion])

has_inc = df['Full_Text'].str.contains(inc_pattern, case=False, na=False, regex=True)
has_exc = df['Full_Text'].str.contains(exc_pattern, case=False, na=False, regex=True)
matches = df[has_inc & ~has_exc]

matches.to_csv('open_fulltext_full.csv', index=False)
print(f"\nOpen fulltext (BAK filtered): {len(matches)}/{len(df)}")
display(matches[['Title', 'URL']])

In [ ]:
"""
4. CLOSED TENDERS - FULL TEXT - FULL BAK KEYWORDS
"""
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/home")
time.sleep(3)

try:
    driver.find_element(By.LINK_TEXT, "Login").click()
    time.sleep(2)
    driver.find_element(By.ID, "username").send_keys("jansencedric37@gmail.com")
    driver.find_element(By.ID, "password").send_keys("Bully1245??")
    driver.find_element(By.NAME, "submit").click()
    time.sleep(5)
except:
    pass

tenders = []
page = 1
while page <= 20:
    driver.get(f"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/calls-for-tenders?pageNumber={page}&status=31094501")
    time.sleep(8)
    elems = driver.find_elements(By.CSS_SELECTOR, "div.ng-star-inserted")
    found = []
    for e in elems:
        txt = e.text.strip()
        if txt and len(txt) > 30:
            try:
                link = e.find_element(By.TAG_NAME, "a").get_attribute("href")
                if link and link.startswith('http'):
                    found.append({'Title': txt.split('\n')[0], 'Full_Text': txt, 'URL': link})
            except: 
                pass
    if not found: 
        break
    tenders.extend(found)
    print(f"Page {page}: {len(found)}")
    page += 1
    time.sleep(3)

driver.quit()

df = pd.DataFrame(tenders).drop_duplicates(subset='URL')

inclusion = ['study', 'Studie', 'analysis', 'Analyse', 'economy', 'Wirtschaft', 'Ökonomie', 
             'economic', 'benchmarking', 'Wirtschaftsberatung', 'market research', 'Marktforschung',
             'statistics', 'Statistiken', 'survey', 'Umfragen', 'index', 'Bundesamt für Statistik',
             'BKS', 'SECO', 'Staatssekretariat für Wirtschaft', 'Kanton Zürich', 'Kanton Luzern',
             'Bundesamt für', 'Regionen', 'Wirtschaftsforschung', 'Dienstleistung', 'Ausschreibung']

exclusion = ['construction', 'Bau', 'health care', 'Gesundheit', 'transport', 'Verkehr',
             'mobility', 'Mobilität', 'sport', 'culture', 'Kultur', 'street', 'Strasse',
             'infrastructure', 'Infrastruktur', 'process', 'Prozess']

inc_pattern = '|'.join([rf'\b{k}\b' for k in inclusion])
exc_pattern = '|'.join([rf'\b{k}\b' for k in exclusion])

has_inc = df['Full_Text'].str.contains(inc_pattern, case=False, na=False, regex=True)
has_exc = df['Full_Text'].str.contains(exc_pattern, case=False, na=False, regex=True)
matches = df[has_inc & ~has_exc]

matches.to_csv('closed_fulltext_full.csv', index=False)
print(f"\nClosed fulltext (BAK filtered): {len(matches)}/{len(df)}")
display(matches[['Title', 'URL']])

In [10]:
"""
EU Portal - Closed Tenders (Keyword Filtering Only) (Draft)
"""

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time

BAK_INCLUSION = [
    'study', 'analysis', 'economy', 'economic', 'benchmarking', 
    'market research', 'statistics', 'survey', 'index', 'economic research',
    'Studie', 'Analyse', 'Wirtschaft', 'Marktforschung', 'Wirtschaftsforschung',
    'Bundesamt für Statistik', 'SECO', 'Bundesamt für'
]

BAK_EXCLUSION = [
    'construction', 'health care', 'transport', 'mobility', 'sport', 
    'culture', 'street', 'infrastructure',
    'Bau', 'Gesundheit', 'Verkehr', 'Mobilität', 'Kultur', 'Strasse', 'Infrastruktur'
]

def scrape_closed_no_cpv():
    
    print("EU PORTAL - CLOSED TENDERS (KEYWORD FILTERING)")
    print("="*70)
    print("Scraping ALL closed tenders since 2018, then filtering by keywords")
    print("This will take 30-60 minutes...\n")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    
    try:
        # Login
        driver.get("https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/home")
        time.sleep(3)
        try:
            driver.find_element(By.LINK_TEXT, "Login").click()
            time.sleep(2)
            driver.find_element(By.ID, "username").send_keys("jansencedric37@gmail.com")
            driver.find_element(By.ID, "password").send_keys("Bully1245??")
            driver.find_element(By.NAME, "submit").click()
            time.sleep(5)
        except:
            pass
        
        # Scrape closed tenders - NO CPV filter
        base_url = "https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/calls-for-tenders?order=DESC&pageNumber={}&pageSize=50&sortBy=startDate&isExactMatch=true&status=31094501&startDate=01/01/2018"
        
        all_tenders = []
        page = 1
        max_pages = 50  # Limit to 2500 tenders for safety
        
        while page <= max_pages:
            url = base_url.format(page)
            driver.get(url)
            time.sleep(8)
            
            elements = driver.find_elements(By.CSS_SELECTOR, "div.ng-star-inserted")
            
            page_tenders = []
            for elem in elements:
                text = elem.text.strip()
                if text and len(text) > 30:
                    try:
                        link = elem.find_element(By.TAG_NAME, "a").get_attribute("href")
                        if link and link.startswith('http'):
                            page_tenders.append({'Title': text[:80], 'Full_Text': text, 'URL': link})
                    except:
                        continue
            
            if not page_tenders:
                print(f"Page {page}: No more results")
                break
            
            all_tenders.extend(page_tenders)
            print(f"Page {page}: {len(page_tenders)} tenders (Total: {len(all_tenders)})")
            
            page += 1
            time.sleep(3)
        
        driver.quit()
        
        print(f"\n{'='*70}")
        print(f"TOTAL SCRAPED: {len(all_tenders)} closed tenders")
        print("="*70 + "\n")
        
        df = pd.DataFrame(all_tenders)
        
        # Apply BAK filtering
        inclusion_pattern = '|'.join([rf'\b{k}\b' for k in BAK_INCLUSION])
        has_inclusion = df['Full_Text'].str.contains(inclusion_pattern, case=False, na=False, regex=True)
        
        exclusion_pattern = '|'.join([rf'\b{k}\b' for k in BAK_EXCLUSION])
        has_exclusion = df['Full_Text'].str.contains(exclusion_pattern, case=False, na=False, regex=True)
        
        relevant = df[has_inclusion & ~has_exclusion].copy()
        
        print(f"After BAK filtering: {len(relevant)} relevant tenders\n")
        
        if not relevant.empty:
            matched = []
            for idx, row in relevant.iterrows():
                text_lower = row['Full_Text'].lower()
                matches = [k for k in BAK_INCLUSION if k.lower() in text_lower]
                matched.append(', '.join(matches[:5]) if matches else 'None')
            
            relevant['Matched_Keywords'] = matched
            
            display(relevant[['Title', 'Matched_Keywords', 'URL']].head(20))
            relevant.to_csv('eu_closed_keyword_filtered.csv', index=False)
            print(f"Saved: eu_closed_keyword_filtered.csv ({len(relevant)} tenders)")
        
        return relevant
        
    except Exception as e:
        print(f"Error: {e}")
        driver.quit()
        return pd.DataFrame()

# Run
closed_keyword = scrape_closed_no_cpv()

EU PORTAL - CLOSED TENDERS (KEYWORD FILTERING)
Scraping ALL closed tenders since 2018, then filtering by keywords
This will take 30-60 minutes...



WebDriverException: Message: Service /Users/cedricjansen/.wdm/drivers/chromedriver/mac64/140.0.7339.207/chromedriver-mac-arm64/chromedriver unexpectedly exited. Status code was: -9


In [9]:
"""
EU Portal - Closed Tenders with Full Details (Final Version)
"""

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time

BAK_INCLUSION = [
    'study', 'analysis', 'economy', 'economic', 'benchmarking', 
    'market research', 'statistics', 'survey', 'index', 'economic research',
    'Studie', 'Analyse', 'Wirtschaft', 'Marktforschung', 'Wirtschaftsforschung',
    'Bundesamt für Statistik', 'SECO', 'Bundesamt für'
]

BAK_EXCLUSION = [
    'construction', 'health care', 'transport', 'mobility', 'sport', 
    'culture', 'street', 'infrastructure',
    'Bau', 'Gesundheit', 'Verkehr', 'Mobilität', 'Kultur', 'Strasse', 'Infrastruktur'
]

def extract_tender_fields(driver, url):
    """Extract full tender details"""
    try:
        driver.get(url)
        time.sleep(5)
        
        text = driver.find_element(By.TAG_NAME, "body").text
        lines = [l.strip() for l in text.split('\n') if l.strip()]
        
        fields = {
            'Procedure_ID': '',
            'Description': '',
            'Procedure_Type': '',
            'Budget': '',
            'Lead_Authority': '',
            'CPV_Code': '',
            'Contract_Duration': '',
            'Deadline': ''
        }
        
        for i, line in enumerate(lines):
            if i+1 >= len(lines):
                continue
            next_line = lines[i+1]
            
            if line == 'Procedure identifier':
                fields['Procedure_ID'] = next_line
            elif line == 'Description' and len(next_line) > 50:
                fields['Description'] = next_line[:250]
            elif line == 'Procedure type':
                fields['Procedure_Type'] = next_line
            elif line == 'Estimated total value':
                fields['Budget'] = next_line
            elif line == 'Lead contracting authority':
                fields['Lead_Authority'] = next_line
            elif 'Main classification (CPV)' in line:
                fields['CPV_Code'] = next_line
            elif line == 'Maximum contract duration':
                fields['Contract_Duration'] = next_line
            elif 'Deadline for receipt of tenders' in line:
                fields['Deadline'] = next_line
        
        return fields
    except:
        return None

def scrape_closed_with_details():
    
    print("EU PORTAL - CLOSED TENDERS WITH FULL DETAILS")
    print("="*70)
    print("Scraping closed tenders since 2018, then extracting full details\n")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    
    try:
        # Login
        driver.get("https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/home")
        time.sleep(3)
        try:
            driver.find_element(By.LINK_TEXT, "Login").click()
            time.sleep(2)
            driver.find_element(By.ID, "username").send_keys("jansencedric37@gmail.com")
            driver.find_element(By.ID, "password").send_keys("Bully1245??")
            driver.find_element(By.NAME, "submit").click()
            time.sleep(5)
        except:
            pass
        
        # Scrape closed tender list
        base_url = "https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/calls-for-tenders?order=DESC&pageNumber={}&pageSize=50&sortBy=startDate&isExactMatch=true&status=31094501&startDate=01/01/2018"
        
        all_tenders = []
        page = 1
        max_pages = 20  # Limit for initial run
        
        print("Step 1: Collecting tender list...")
        while page <= max_pages:
            url = base_url.format(page)
            driver.get(url)
            time.sleep(8)
            
            elements = driver.find_elements(By.CSS_SELECTOR, "div.ng-star-inserted")
            
            page_tenders = []
            for elem in elements:
                text = elem.text.strip()
                if text and len(text) > 30:
                    try:
                        link = elem.find_element(By.TAG_NAME, "a").get_attribute("href")
                        if link and link.startswith('http'):
                            page_tenders.append({'Title': text[:80], 'Full_Text': text, 'URL': link})
                    except:
                        continue
            
            if not page_tenders:
                print(f"  Page {page}: No more results")
                break
            
            all_tenders.extend(page_tenders)
            print(f"  Page {page}: {len(page_tenders)} tenders (Total: {len(all_tenders)})")
            
            page += 1
            time.sleep(3)
        
        print(f"\nTotal scraped: {len(all_tenders)} tenders")
        
        # BAK filtering
        df = pd.DataFrame(all_tenders)
        inclusion_pattern = '|'.join([rf'\b{k}\b' for k in BAK_INCLUSION])
        has_inclusion = df['Full_Text'].str.contains(inclusion_pattern, case=False, na=False, regex=True)
        
        exclusion_pattern = '|'.join([rf'\b{k}\b' for k in BAK_EXCLUSION])
        has_exclusion = df['Full_Text'].str.contains(exclusion_pattern, case=False, na=False, regex=True)
        
        relevant = df[has_inclusion & ~has_exclusion]
        
        print(f"BAK-relevant: {len(relevant)} tenders\n")
        
        if relevant.empty:
            print("No BAK-relevant tenders found")
            driver.quit()
            return pd.DataFrame()
        
        # Extract full details for relevant tenders
        print("Step 2: Extracting full details for relevant tenders...\n")
        
        results = []
        for idx, row in relevant.iterrows():
            print(f"  {len(results)+1}/{len(relevant)}: Extracting details...")
            
            fields = extract_tender_fields(driver, row['URL'])
            if fields:
                results.append({
                    'Title': row['Title'],
                    'Procedure_ID': fields['Procedure_ID'],
                    'Budget': fields['Budget'],
                    'Procedure_Type': fields['Procedure_Type'],
                    'Deadline': fields['Deadline'],
                    'Contract_Duration': fields['Contract_Duration'],
                    'Lead_Authority': fields['Lead_Authority'],
                    'CPV_Code': fields['CPV_Code'],
                    'Description': fields['Description'],
                    'URL': row['URL']
                })
            
            time.sleep(2)
        
        driver.quit()
        
        final_df = pd.DataFrame(results)
        
        print(f"\n{'='*70}")
        print(f"COMPLETE: {len(final_df)} closed BAK-relevant tenders with details")
        print("="*70 + "\n")
        
        if not final_df.empty:
            display(final_df[['Title', 'Budget', 'Deadline', 'Procedure_Type']])
            final_df.to_csv('eu_closed_tenders_detailed.csv', index=False)
            print("\nSaved: eu_closed_tenders_detailed.csv")
        
        return final_df
        
    except Exception as e:
        print(f"Error: {e}")
        driver.quit()
        return pd.DataFrame()

# Run
closed_detailed = scrape_closed_with_details()

EU PORTAL - CLOSED TENDERS WITH FULL DETAILS
Scraping closed tenders since 2018, then extracting full details



WebDriverException: Message: Service /Users/cedricjansen/.wdm/drivers/chromedriver/mac64/140.0.7339.207/chromedriver-mac-arm64/chromedriver unexpectedly exited. Status code was: -9


In [ ]:
"""
BSV SCRAPER WITH KEYWORD MATCHING (Draft)
"""

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# BAK Keywords
BAK_INCLUSION = [
    'study', 'analysis', 'economy', 'economic', 'statistics', 'survey',
    'Studie', 'Analyse', 'Wirtschaft', 'Statistik', 'Umfragen',
    'Forschung', 'research', 'Bundesamt', 'Sozialversicherung'
]

def scrape_bsv_with_keywords():
    """
    BSV scraper with matched keyword identification
    """
    print("BSV SCRAPER WITH KEYWORD MATCHING")
    print("="*70)
    
    url = "https://www.bsv.admin.ch/bsv/de/home/publikationen-und-service/forschung/ausschreibungen.html"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=15)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            print("Page loaded successfully\n")
            
            tenders = []
            all_links = soup.find_all('a', href=True)
            
            for link in all_links:
                href = link.get('href', '')
                text = link.text.strip()
                
                if '.pdf' in href.lower() or 'ausschreibung' in text.lower():
                    
                    # Make full URL
                    if href.startswith('http'):
                        full_url = href
                    elif href.startswith('/'):
                        full_url = f"https://www.bsv.admin.ch{href}"
                    else:
                        full_url = f"https://www.bsv.admin.ch/bsv/de/home/publikationen-und-service/forschung/{href}"
                    
                    # Extract date
                    date_match = re.search(r'(\d{2}\.\d{2}\.\d{4})', text)
                    date = date_match.group(1) if date_match else 'N/A'
                    
                    # Find matched keywords
                    text_lower = text.lower()
                    matches = [k for k in BAK_INCLUSION if k.lower() in text_lower]
                    
                    tenders.append({
                        'Platform': 'BSV',
                        'Title': text[:100] if text else 'BSV Document',
                        'Matched_Keywords': ', '.join(matches[:5]) if matches else 'None',
                        'Date': date,
                        'Type': 'PDF' if '.pdf' in href else 'Page',
                        'URL': full_url
                    })
            
            df = pd.DataFrame(tenders)
            
            print(f"Found {len(df)} documents")
            print(f"  PDFs: {len(df[df['Type'] == 'PDF']) if not df.empty else 0}\n")
            
            if not df.empty:
                # Display clean columns
                display(df[['Platform', 'Title', 'Matched_Keywords', 'Date', 'URL']])
                
                # Save
                df[['Platform', 'Title', 'Matched_Keywords', 'Date', 'URL']].to_csv('bsv_results.csv', index=False)
                print("\nSaved: bsv_results.csv")
            
            return df
        else:
            print(f"Failed: {response.status_code}")
            return pd.DataFrame()
            
    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame()

# Run
bsv_data = scrape_bsv_with_keywords()

In [ ]:
"""
BSV Scraper with BAK Methodology (final version)
"""

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Complete BAK Keywords
BAK_INCLUSION = [
    'study', 'analysis', 'economy', 'economic', 'benchmarking', 
    'economic consulting', 'market research', 'statistics', 'survey',
    'index', 'regions', 'economic research',
    'Studie', 'Analyse', 'Wirtschaft', 'Ökonomie', 'Wirtschaftsberatung',
    'Marktforschung', 'Wirtschaftsforschung', 'Umfragen', 'Statistiken',
    'Dienstleistung', 'Ausschreibung', 'Verbraucherindex', 'Regionen',
    'Bundesamt für Statistik', 'BKS', 'Office fédéral de la statistique',
    'SECO', 'Staatssekretariat für Wirtschaft',
    'Kanton Zürich', 'Kanton Luzern', 'BBL', 'Bundesamt für',
    'Forschung', 'research'
]

BAK_EXCLUSION = [
    'construction', 'health care', 'transport', 'mobility', 'sport', 
    'culture', 'street', 'infrastructure', 'IT',
    'Bau', 'Gesundheit', 'Verkehr', 'Mobilität', 
    'Kultur', 'Strasse', 'Infrastruktur', 'Prozess'
]

def scrape_bsv():
    
    print("BSV SCRAPER WITH BAK FILTERING")
    print("="*70)
    
    url = "https://www.bsv.admin.ch/bsv/de/home/publikationen-und-service/forschung/ausschreibungen.html"
    headers = {'User-Agent': 'Mozilla/5.0'}
    
    try:
        response = requests.get(url, headers=headers, timeout=15)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Check for no tenders message
        if 'keine Projekte ausgeschrieben' in soup.get_text():
            print("No active tenders on BSV (currently 0 posted)")
            print("BSV posts 2-5 research tenders per year\n")
            return pd.DataFrame()
        
        # Find tender links
        tenders = []
        for link in soup.find_all('a', href=True):
            href = link['href']
            text = link.text.strip()
            
            if '.pdf' in href.lower() or '/dam/bsv/' in href:
                full_url = href if href.startswith('http') else f"https://www.bsv.admin.ch{href}"
                
                date_match = re.search(r'(\d{2}\.\d{2}\.\d{4})', text)
                date = date_match.group(1) if date_match else 'N/A'
                
                tenders.append({
                    'Platform': 'BSV',
                    'Title': text[:100] if text else 'BSV Document',
                    'Full_Text': text,
                    'Date': date,
                    'URL': full_url
                })
        
        if not tenders:
            print("No documents found\n")
            return pd.DataFrame()
        
        df = pd.DataFrame(tenders)
        print(f"Found {len(df)} documents")
        
        # Apply BAK filtering
        inclusion_pattern = '|'.join([rf'\b{k}\b' for k in BAK_INCLUSION])
        has_inclusion = df['Full_Text'].str.contains(inclusion_pattern, case=False, na=False, regex=True)
        
        exclusion_pattern = '|'.join([rf'\b{k}\b' for k in BAK_EXCLUSION])
        has_exclusion = df['Full_Text'].str.contains(exclusion_pattern, case=False, na=False, regex=True)
        
        filtered = df[has_inclusion & ~has_exclusion]
        print(f"After BAK filter: {len(filtered)} relevant\n")
        
        # Add matched keywords
        if not filtered.empty:
            matched = []
            for idx, row in filtered.iterrows():
                text_lower = row['Full_Text'].lower()
                matches = [k for k in BAK_INCLUSION if k.lower() in text_lower]
                matched.append(', '.join(matches[:5]) if matches else 'None')
            
            filtered['Matched_Keywords'] = matched
            
            display(filtered[['Platform', 'Title', 'Matched_Keywords', 'Date', 'URL']])
            filtered[['Platform', 'Title', 'Matched_Keywords', 'Date', 'URL']].to_csv('bsv_results.csv', index=False)
            print("Saved: bsv_results.csv")
        
        return filtered
        
    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame()

# Run
bsv_data = scrape_bsv()

In [ ]:
"""
SAVE RESULTS TO FILES
=====================
Export data to CSV and JSON for later use
"""

def save_results(df, filename_prefix):
    """Save DataFrame to CSV and JSON"""
    
    if df.empty:
        print(f"⚠️  No data to save for {filename_prefix}")
        return
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # CSV
    csv_file = f"{filename_prefix}_{timestamp}.csv"
    df.to_csv(csv_file, index=False, encoding='utf-8')
    print(f"💾 Saved: {csv_file}")
    
    # JSON
    json_file = f"{filename_prefix}_{timestamp}.json"
    df.to_json(json_file, orient='records', indent=2)
    print(f"💾 Saved: {json_file}")
    
    return csv_file, json_file

print("💾 SAVING RESULTS")
print("="*70 + "\n")

# Save complete BAK search results
if 'bak_complete' in locals() and not bak_complete.empty:
    save_results(bak_complete, 'bak_tenders_complete')

# Save BSV results
if 'bsv_results' in locals() and not bsv_results.empty:
    save_results(bsv_results, 'bsv_documents')

print("\n✅ Export complete!")

In [86]:
# Company's EU Portal tender titles
company_eu_titles = eu_only['Thema'].tolist()

# Your scraped titles
your_titles = list(open_title['Title']) + list(open_full['Title']) + list(closed_title['Title']) + list(closed_full['Title'])
your_titles = list(set(your_titles))  # Remove duplicates

print(f"Company found: {len(company_eu_titles)} EU Portal tenders")
print(f"You scraped: {len(your_titles)} unique tenders\n")

# Check for matches
print("Checking for overlaps...")
matches = 0
for company_title in company_eu_titles:
    company_title_clean = str(company_title).lower()[:50]
    for your_title in your_titles:
        your_title_clean = str(your_title).lower()[:50]
        if company_title_clean in your_title_clean or your_title_clean in company_title_clean:
            print(f"MATCH: {company_title[:60]}")
            matches += 1
            break

print(f"\n{'='*70}")
print(f"Matches found: {matches}/{len(company_eu_titles)}")
print(f"Company tenders you DIDN'T capture: {len(company_eu_titles) - matches}")
print("="*70)

# Show most recent company tender (should be in your open results)
print("\nMost recent company EU tender:")
recent = eu_only.iloc[-1]
print(f"  Title: {recent['Thema']}")
print(f"  Date: {recent['Meldung Webseite']}")
print(f"  Deadline: {recent['Eingabefrist']}")

Company found: 16 EU Portal tenders
You scraped: 33 unique tenders

Checking for overlaps...

Matches found: 0/16
Company tenders you DIDN'T capture: 16

Most recent company EU tender:
  Title: Jährlicher wissenschaftlicher Beitrag zu den Tourismusberichten 2024-2027 
  Date: 2024-08-19 00:00:00
  Deadline: 2024-09-17 00:00:00


In [118]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time
import re

# URL of the ESPON procurements page
BASE_URL = "https://www.espon.eu/engage/procurements"

# Payload: we will fetch by date range (month-based)
start_year = 2018
today = datetime.date.today()

all_notices = []

# Loop through months/years from 2018 to now
for year in range(start_year, today.year + 1):
    start_month = 1
    end_month = 12
    if year == today.year:
        end_month = today.month  # up to current month
    
    for month in range(start_month, end_month + 1):
        start_date = datetime.date(year, month, 1)
        if year == today.year and month == today.month:
            end_date = today
        else:
            # last day of month
            next_month = datetime.date(year + int(month/12), (month % 12) + 1, 1)
            end_date = next_month - datetime.timedelta(days=1)
        print(f"Processing {start_date} to {end_date} ...")
        
        # Build URL with date filters (depends on page structure)
        # For simplicity, we search in the page content directly
        # You may need to tweak the URL or request parameters
        params = {
            # Add request params if needed
            # e.g., 'dateFrom': start_date.strftime('%Y-%m-%d'), ...
        }

        # Access the page
        response = requests.get(BASE_URL)
        if response.status_code != 200:
            print(f"Failed to load page for {year}-{month}")
            continue
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find tender cards/links, depend on actual page structure
        tender_cards = soup.find_all('div', class_=re.compile('.*view.*|.*item.*|.*card.*', re.I))
        links = soup.find_all('a', href=True)
        
        for a in links:
            href = a['href']
            text = a.get_text(strip=True)
            # Check if link points to a procurement detail
            if "/engage/procurements/" in href and text and len(text) > 10:
                full_url = href
                if not href.startswith('http'):
                    # Fix relative links
                    if href.startswith('/'):
                        full_url = f"https://www.espon.eu{href}"
                    else:
                        full_url = f"https://www.espon.eu/engage/procurements/{href}"
                # Extract some info (just as example)
                # You might need to extract deadline, etc., from surrounding text
                # For demo, we'll store link and text
                all_notices.append({
                    'title': text,
                    'url': full_url,
                    'date_posted': start_date.strftime('%Y-%m-%d'),  # approximate
                    # Add more fields if you parse them from HTML
                })

        time.sleep(1)  # be polite to server

# --- After scraping, convert to DataFrame ---
df = pd.DataFrame(all_notices)

# --- Now, apply keyword filters ---
relevant_keywords = [
    "study", "studie", "analysis", "analyse", "economy", "wirtschaft", "ökonomie",
    "benchmarking", "markt", "forschung", "statistiken", "umfragen"
]
exclude_keywords = [
    "construction", "bau", "health", "gesundheit", "transport", "verkehr",
    "mobility", "mobilität", "sport", "culture", "kultur", "street", "strasse"
]

def filter_keywords(text):
    text = text.lower()
    if any(kw.lower() in text for kw in relevant_keywords) and not any(ekw.lower() in text for ekw in exclude_keywords):
        return True
    return False

# Filter notices
df['relevant'] = df['title'].apply(lambda x: filter_keywords(str(x)))

# Separate relevant and non-relevant
df


Processing 2018-01-01 to 2018-01-31 ...
Failed to load page for 2018-1
Processing 2018-02-01 to 2018-02-28 ...
Failed to load page for 2018-2
Processing 2018-03-01 to 2018-03-31 ...
Failed to load page for 2018-3
Processing 2018-04-01 to 2018-04-30 ...
Failed to load page for 2018-4
Processing 2018-05-01 to 2018-05-31 ...
Failed to load page for 2018-5
Processing 2018-06-01 to 2018-06-30 ...
Failed to load page for 2018-6
Processing 2018-07-01 to 2018-07-31 ...
Failed to load page for 2018-7
Processing 2018-08-01 to 2018-08-31 ...
Failed to load page for 2018-8
Processing 2018-09-01 to 2018-09-30 ...
Failed to load page for 2018-9
Processing 2018-10-01 to 2018-10-31 ...
Failed to load page for 2018-10
Processing 2018-11-01 to 2018-11-30 ...
Failed to load page for 2018-11
Processing 2018-12-01 to 2018-12-31 ...
Failed to load page for 2018-12
Processing 2019-01-01 to 2019-01-31 ...
Failed to load page for 2019-1
Processing 2019-02-01 to 2019-02-28 ...
Failed to load page for 2019-2
Pro

KeyError: 'title'

In [121]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from langdetect import detect

# URL of the listings page
LIST_URL = "https://www.espon.eu/engage/procurements"

# Function to get all tender URLs from the main page
def get_tender_urls():
    response = requests.get(LIST_URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = []
    for a in soup.find_all('a', href=True):
        href = a['href']
        if "/engage/procurements/" in href:
            if href.startswith('/'):
                links.append("https://www.espon.eu" + href)
            elif href.startswith('http'):
                links.append(href)
    print(f"Found {len(links)} tenders.")
    return links

# Function to scrape each individual tender page
def scrape_tender(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Critically, update the class names/tag selections as per actual page layout
    try:
        full_text = soup.find('div', class_='description').get_text(strip=True)
    except:
        full_text = 'N/A'

    try:
        org_name = soup.find('div', class_='organization').get_text(strip=True)
    except:
        org_name = 'N/A'

    try:
        title = soup.find('h1').get_text(strip=True)
    except:
        title = 'N/A'

    try:
        cpv_match = re.search(r'\d{8}', full_text)
        cpv = cpv_match.group() if cpv_match else 'N/A'
    except:
        cpv = 'N/A'

    try:
        pub_date = soup.find('div', class_='publication-date').get_text(strip=True)
    except:
        pub_date = 'N/A'

    # Deadlines
    deadline_submit = 'N/A'
    deadline_info = 'N/A'
    for span in soup.find_all('span'):
        txt = span.get_text().lower()
        if 'deadline' in txt:
            date_match = re.search(r'\d{1,2}\.\d{1,2}\.\d{4}', txt)
            if date_match:
                date_str = date_match.group()
                if 'submit' in txt:
                    deadline_submit = date_str
                elif 'additional' in txt or 'info' in txt:
                    deadline_info = date_str

    # Procurement value
    price = 'N/A'
    # Adjust class or method to extract price if available

    # Language detection
    try:
        lang_code = detect(full_text)
        lang_map = {'de': 'German', 'en': 'English', 'fr': 'French', 'it': 'Italian'}
        language = lang_map.get(lang_code, 'unknown')
    except:
        language = 'unknown'

    return {
        'Title': title,
        'CPV': cpv,
        'Full Text': full_text,
        'Organization': org_name,
        'Publication Date': pub_date,
        'Deadline for Submission': deadline_submit,
        'Deadline for Info': deadline_info,
        'Procurement Value': price,
        'Language': language,
        'URL': url
    }

def main():
    # 1. Get list of tenders
    urls = get_tender_urls()
    data_list = []

    # 2. Scrape each
    for url in urls:
        print(f"Scraping {url}")
        data = scrape_tender(url)
        if data:  # add only if data was successfully scraped
            data_list.append(data)
        time.sleep(1)

    # 3. Convert to DataFrame
    df = pd.DataFrame(data_list)

    # 4. Save to Excel in sheets per language (if data exists)
    if not df.empty:
        with pd.ExcelWriter("tenders_by_language.xlsx") as writer:
            for lang in df['Language'].dropna().unique():
                df[df['Language'] == lang].to_excel(writer, sheet_name=lang, index=False)
    else:
        print("No data scraped, skipping Excel save!")

    # 5. Save all in CSV
    df.to_csv("all_tenders.csv", index=False)
    print("All data saved to 'all_tenders.csv' and 'tenders_by_language.xlsx'.")

if __name__ == "__main__":
    main()


Found 0 tenders.
No data scraped, skipping Excel save!
All data saved to 'all_tenders.csv' and 'tenders_by_language.xlsx'.


In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time

INCLUSION = ['study', 'Studie', 'analysis', 'Analyse', 'economy', 'Wirtschaft', 'Ökonomie', 
             'economic', 'benchmarking', 'Wirtschaftsberatung', 'market research', 'Marktforschung',
             'statistics', 'Statistiken', 'survey', 'Umfragen', 'index', 'Bundesamt für Statistik',
             'BKS', 'SECO', 'Staatssekretariat für Wirtschaft', 'Kanton Zürich', 'Kanton Luzern',
             'Bundesamt für', 'Regionen', 'Wirtschaftsforschung', 'Dienstleistung', 'Ausschreibung']

EXCLUSION = ['construction', 'Bau', 'health care', 'Gesundheit', 'transport', 'Verkehr',
             'mobility', 'Mobilität', 'sport', 'culture', 'Kultur', 'street', 'Strasse',
             'infrastructure', 'Infrastruktur', 'process', 'Prozess']

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/home")
time.sleep(5)

tenders = []
for page in range(1, 21):
    driver.get(f"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/calls-for-tenders?pageNumber={page}&status=31094502")
    time.sleep(8)
    for e in driver.find_elements(By.CSS_SELECTOR, "div.ng-star-inserted"):
        txt = e.text.strip()
        if txt and len(txt) > 30:
            try:
                link = e.find_element(By.TAG_NAME, "a").get_attribute("href")
                if link: tenders.append({'Title': txt.split('\n')[0], 'Full_Text': txt, 'URL': link})
            except: pass
    print(f"Page {page}: {len(tenders)} total")

driver.quit()

print("\nGetting description")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
for idx in df.index:
    driver.get(df.at[idx, 'URL'])
    time.sleep(5)
    try:
        lines = driver.find_element(By.TAG_NAME, "body").text.split('\n')
        desc = next((lines[i+1][:500] for i, l in enumerate(lines[:-1]) if 'Description' in l and len(lines[i+1]) > 50), "N/A")
    except:
        df.at[idx, 'Description'] = "N/A"
    print(f"{idx+1}/{len(df)}")
    
driver.quit()
df = pd.DataFrame(tenders).drop_duplicates(subset='URL').reset_index(drop=True)
inc_pattern = '|'.join([rf'\b{k}\b' for k in INCLUSION])
exc_pattern = '|'.join([rf'\b{k}\b' for k in EXCLUSION])
filtered = df[df['Full_Text'].str.contains(inc_pattern, case=False, na=False, regex=True) & 
              ~df['Full_Text'].str.contains(exc_pattern, case=False, na=False, regex=True)]

filtered.to_csv('eu_open_current.csv', index=False, encoding='utf-8-sig')
print(f"Saved {len(filtered)}/{len(df)} open tenders")

Page 1: 100 total
Page 2: 200 total
Page 3: 300 total
Page 4: 400 total
Page 5: 500 total
Page 6: 600 total
Page 7: 700 total
Page 8: 800 total
Page 9: 828 total
Page 10: 828 total
Page 11: 828 total
Page 12: 828 total
Page 13: 828 total
Page 14: 828 total
Page 15: 828 total
Page 16: 828 total
Page 17: 828 total
Page 18: 828 total
Page 19: 828 total
Page 20: 828 total

Getting description
1/414
3/414
5/414
7/414
9/414
11/414
13/414
15/414
17/414
19/414
21/414
23/414
25/414
27/414
29/414
31/414
33/414
35/414
37/414
39/414
41/414
43/414
45/414
47/414
49/414
51/414
53/414
55/414
57/414
59/414
61/414
63/414
65/414
67/414
69/414
71/414
73/414
75/414
77/414
79/414
81/414
83/414
85/414
87/414
89/414
91/414
93/414
95/414
97/414
99/414
101/414
103/414
105/414
107/414
109/414
111/414
113/414
115/414
117/414
119/414
121/414
123/414
125/414
127/414
129/414
131/414
133/414
135/414
137/414
139/414
141/414
143/414
145/414
147/414
149/414
151/414
153/414
155/414
157/414
159/414
161/414
163/414
165/414

In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time

INCLUSION = ['study', 'Studie', 'analysis', 'Analyse', 'economy', 'Wirtschaft', 'Ökonomie', 
             'economic', 'benchmarking', 'Wirtschaftsberatung', 'market research', 'Marktforschung',
             'statistics', 'Statistiken', 'survey', 'Umfragen', 'index', 'Bundesamt für Statistik',
             'BKS', 'SECO', 'Staatssekretariat für Wirtschaft', 'Kanton Zürich', 'Kanton Luzern',
             'Bundesamt für', 'Regionen', 'Wirtschaftsforschung', 'Dienstleistung', 'Ausschreibung']

EXCLUSION = ['construction', 'Bau', 'health care', 'Gesundheit', 'transport', 'Verkehr',
             'mobility', 'Mobilität', 'sport', 'culture', 'Kultur', 'street', 'Strasse',
             'infrastructure', 'Infrastruktur', 'process', 'Prozess']


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/home")
time.sleep(5)

tenders = []
for page in range(1, 51):
    driver.get(f"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/calls-for-tenders?pageNumber={page}&status=31094501&startDate=01/01/2018")
    time.sleep(8)
    found = []
    for e in driver.find_elements(By.CSS_SELECTOR, "div.ng-star-inserted"):
        txt = e.text.strip()
        if txt and len(txt) > 30:
            try:
                link = e.find_element(By.TAG_NAME, "a").get_attribute("href")
                if link and link.startswith('http'):
                    found.append({'Title': txt.split('\n')[0], 'Full_Text': txt, 'URL': link})
            except: pass
    tenders.extend(found)
    print(f"Page {page}: {len(tenders)} total")
    
driver.quit()

df = pd.DataFrame(tenders).drop_duplicates(subset='URL').reset_index(drop=True)
print(f"Unique tenders: {len(df)}")

inc_pattern = '|'.join([rf'\b{k}\b' for k in INCLUSION])
exc_pattern = '|'.join([rf'\b{k}\b' for k in EXCLUSION])
filtered = df[df['Full_Text'].str.contains(inc_pattern, case=False, na=False, regex=True) & 
              ~df['Full_Text'].str.contains(exc_pattern, case=False, na=False, regex=True)].reset_index(drop=True)

print(f"After filtering: {len(filtered)}")
filtered.to_csv('eu_closed_2018-now.csv', index=False, encoding='utf-8-sig')
print(f"Saved {len(filtered)} relevant closed tenders")



Page 1: 100 total
Page 2: 200 total
Page 3: 300 total
Page 4: 366 total
Page 5: 366 total
Page 6: 366 total
Page 7: 366 total
Page 8: 366 total
Page 9: 366 total
Page 10: 366 total
Page 11: 366 total
Page 12: 366 total
Page 13: 366 total
Page 14: 366 total
Page 15: 366 total
Page 16: 366 total
Page 17: 366 total
Page 18: 366 total
Page 19: 366 total
Page 20: 366 total
Page 21: 366 total
Page 22: 366 total
Page 23: 366 total
Page 24: 366 total
Page 25: 366 total
Page 26: 366 total
Page 27: 366 total
Page 28: 366 total
Page 29: 366 total
Page 30: 366 total
Page 31: 366 total
Page 32: 366 total
Page 33: 366 total
Page 34: 366 total
Page 35: 366 total
Page 36: 366 total
Page 37: 366 total
Page 38: 366 total
Page 39: 366 total
Page 40: 366 total
Page 41: 366 total
Page 42: 366 total
Page 43: 366 total
Page 44: 366 total
Page 45: 366 total
Page 46: 366 total
Page 47: 366 total
Page 48: 366 total
Page 49: 366 total
Page 50: 366 total
Unique tenders: 183
After filtering: 13
Saved 13 relevant c

In [ ]:
import pandas as pd 
from datetime import datetime

df = pd.read_csv('eu_closed_2018-now.csv')

print("="*50)
print("CLOSED TENDERS METADATA (2018-NOW)")
print("="*50)
print(f"Total relevant tenders: {len(df)}")
print(f"Unique URLs: {df['URL'].nunique()}")
print(f"Average title length: {df['Title'].str.len().mean():.1f} characters")
print(f"Date range: 218-01-01 to {datetime.now().strftime('%Y-%m-%d')}")
print(f"Scraped on: {datetime.now().strftime('%Y-%m-%d %H:%M')}")


metadata = pd.DataFrame([{
    'type': 'Closed Tenders',
    'count': len(df),
    'unique_urls': df['URL'].nunique(),
    'avg_title_length': f"{df['Title'].str.len().mean(),
    'date_range': '2018-01-01 to ' + datetime.now().strftime('%Y-%m-%d'),
    'scrape_date': datetime.now().strftime('%Y-%m-%d %H:%M')
}])

metadata.to_csv('eu_closed_metadata.csv', index=False)
print("\nSaved: eu_closed_metadata.csv")

# Here also needs improvement 


In [ ]:
import pandas as pd
from datetime import datetime

df = pd.read_csv('eu_open_current.csv')

print("="*50)
print("OPEN TENDERS METADATA")
print("="*50)
print(f"Total relevant tenders: {len(df)}")
print(f"Unique URLs: {df['URL'].nunique()}")
print(f"Average title length: {df['Title'].str.len().mean():.0f} characters")
print(f"Scraped on: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

metadata = pd.DataFrame([{
    'type': 'open',
    'count': len(df),
    'unique_urls': df['URL'].nunique(),
    'avg_title_length': df['Title'].str.len().mean(),
    'scraped_date': datetime.now()
}])

metadata.to_csv('metadata_open.csv', index=False)
print("\nSaved: metadata_open.csv")


# Needs improvement because this only took 0.5 seconds and it shouldnt take so long

OPEN TENDERS METADATA
Total relevant tenders: 18
Unique URLs: 18
Average title length: 98 characters
Scraped on: 2025-10-06 17:20

Saved: metadata_open.csv


Saved 50 datasets


In [42]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time
from datetime import datetime

INCLUSION = ['study', 'Studie', 'analysis', 'Analyse', 'economy', 'Wirtschaft', 'Ökonomie', 
             'economic', 'benchmarking', 'Wirtschaftsberatung', 'market research', 'Marktforschung',
             'statistics', 'Statistiken', 'survey', 'Umfragen', 'index', 'Bundesamt für Statistik',
             'BKS', 'SECO', 'Staatssekretariat für Wirtschaft', 'Kanton Zürich', 'Kanton Luzern',
             'Bundesamt für', 'Regionen', 'Wirtschaftsforschung', 'Dienstleistung', 'Ausschreibung']

EXCLUSION = ['construction', 'Bau', 'health care', 'Gesundheit', 'transport', 'Verkehr',
             'mobility', 'Mobilität', 'sport', 'culture', 'Kultur', 'street', 'Strasse',
             'infrastructure', 'Infrastruktur', 'process', 'Prozess']

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
tenders = []
page = 1

while page <= 200:
    driver.get(f"https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/calls-for-tenders?pageNumber={page}&status=31094502")
    time.sleep(8)
    
    page_has_2018 = False
    for e in driver.find_elements(By.CSS_SELECTOR, "div.ng-star-inserted"):
        txt = e.text.strip()
        if txt and len(txt) > 30:
            try:
                # Check if text contains a year 2018 or earlier
                if '2018' in txt or '2017' in txt or '2016' in txt:
                    page_has_2018 = True
                
                link = e.find_element(By.TAG_NAME, "a").get_attribute("href")
                if link:
                    tenders.append({'Title': txt.split('\n')[0], 'Full_Text': txt, 'URL': link})
            except: 
                pass
    
    print(f"Page {page}: {len(tenders)} total")
    
    if page_has_2018:
        print("Reached 2018, stopping.")
        break
    
    page += 1

driver.quit()

print(f"\nTotal tenders: {len(tenders)}")
df = pd.DataFrame(tenders).drop_duplicates(subset='URL').reset_index(drop=True)
print(f"Unique tenders: {len(df)}")

# Apply filters
inc_pattern = '|'.join([rf'\b{k}\b' for k in INCLUSION])
exc_pattern = '|'.join([rf'\b{k}\b' for k in EXCLUSION])
filtered = df[df['Full_Text'].str.contains(inc_pattern, case=False, na=False, regex=True) & 
              ~df['Full_Text'].str.contains(exc_pattern, case=False, na=False, regex=True)]

print(f"After filtering: {len(filtered)}")

# Get descriptions
if len(filtered) > 0:
    print(f"\nGetting descriptions for {len(filtered)} relevant tenders...")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    for i, idx in enumerate(filtered.index, 1):
        driver.get(filtered.at[idx, 'URL'])
        time.sleep(5)
        try:
            lines = driver.find_element(By.TAG_NAME, "body").text.split('\n')
            desc = next((lines[i+1][:500] for i, l in enumerate(lines[:-1]) if 'Description' in l and len(lines[i+1]) > 50), "N/A")
            filtered.at[idx, 'Description'] = desc
        except:
            filtered.at[idx, 'Description'] = "N/A"
        print(f"{i}/{len(filtered)} - Done")

    driver.quit()
else:
    print("\nNo relevant tenders found to get descriptions for.")

filtered.to_csv('eu_closed_tenders.csv', index=False, encoding='utf-8-sig')
print(f"\nSaved {len(filtered)} closed tenders")

Page 1: 100 total
Page 2: 200 total
Page 3: 300 total
Page 4: 400 total
Page 5: 500 total
Page 6: 600 total
Page 7: 700 total
Page 8: 800 total
Reached 2018, stopping.

Total tenders: 800
Unique tenders: 400
After filtering: 19

Getting descriptions for 19 relevant tenders...


/var/folders/k7/48dcpx7j1lj9tk5jnj7lzhqc0000gn/T/ipykernel_97354/2757096280.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered.at[idx, 'Description'] = desc


1/19 - Done
2/19 - Done
3/19 - Done
4/19 - Done
5/19 - Done
6/19 - Done
7/19 - Done
8/19 - Done
9/19 - Done
10/19 - Done
11/19 - Done
12/19 - Done
13/19 - Done
14/19 - Done
15/19 - Done
16/19 - Done
17/19 - Done
18/19 - Done
19/19 - Done

Saved 19 closed tenders


In [48]:
import requests
import pandas as pd

INCLUSION = ['study', 'analysis', 'economy', 'economic', 'research', 'statistics', 'survey', 'benchmarking']
EXCLUSION = ['construction', 'health care', 'transport', 'mobility', 'sport', 'culture', 'infrastructure']

# Search data.europa.eu for TED datasets
print("=== Searching for TED procurement data ===")

all_data = []
page = 0

while page < 10:  # Max 10 pages
    response = requests.get('https://data.europa.eu/api/hub/search/search', params={
        'q': 'TED procurement tenders',
        'filter': 'dataset',
        'limit': 100,
        'page': page,
        'sort': 'issued+desc'
    })
    
    if response.status_code != 200:
        break
    
    data = response.json().get('result', {})
    results = data.get('results', [])
    
    if not results:
        break
    
    for item in results:
        title = item.get('title', {})
        desc = item.get('description', {})
        
        # Check all language versions
        text_all = ' '.join([
            str(title.get('en', '')),
            str(title.get('de', '')),
            str(title.get('fr', '')),
            str(title.get('it', '')),
            str(desc.get('en', '')),
            str(desc.get('de', '')),
            str(desc.get('fr', '')),
            str(desc.get('it', ''))
        ]).lower()
        
        # Filter by keywords
        if any(k.lower() in text_all for k in INCLUSION) and not any(k.lower() in text_all for k in EXCLUSION):
            all_data.append({
                'Title_EN': title.get('en', ''),
                'Title_DE': title.get('de', ''),
                'Title_FR': title.get('fr', ''),
                'Title_IT': title.get('it', ''),
                'Description_EN': str(desc.get('en', ''))[:500],
                'Publisher': item.get('publisher', {}).get('name', 'N/A'),
                'Published': item.get('issued', 'N/A'),
                'URL': item.get('landingPage', ['N/A'])[0] if item.get('landingPage') else 'N/A'
            })
    
    print(f"Page {page}: {len(all_data)} relevant datasets")
    page += 1

# Save to Excel
if all_data:
    df = pd.DataFrame(all_data)
    df.to_excel('ted_procurement_datasets.xlsx', index=False, engine='openpyxl')
    print(f"\nSaved {len(all_data)} datasets to ted_procurement_datasets.xlsx")
else:
    print("\nNo relevant datasets found")

=== Searching for TED procurement data ===
Page 0: 18 relevant datasets
Page 1: 34 relevant datasets
Page 2: 36 relevant datasets
Page 3: 50 relevant datasets
Page 4: 53 relevant datasets
Page 5: 53 relevant datasets
Page 6: 74 relevant datasets
Page 7: 86 relevant datasets
Page 8: 99 relevant datasets
Page 9: 128 relevant datasets

Saved 128 datasets to ted_procurement_datasets.xlsx


In [50]:
import pandas as pd

print("=== Processing and saving unfiltered data ===")

all_records = []

for item in datasets_raw:
    title = item.get('title', {})
    desc = item.get('description', {})
    publisher = item.get('publisher', {})
    
    all_records.append({
        'Title_EN': title.get('en', ''),
        'Title_DE': title.get('de', ''),
        'Title_FR': title.get('fr', ''),
        'Title_IT': title.get('it', ''),
        'Description_EN': str(desc.get('en', ''))[:500],
        'Description_DE': str(desc.get('de', ''))[:500],
        'Description_FR': str(desc.get('fr', ''))[:500],
        'Description_IT': str(desc.get('it', ''))[:500],
        'Publisher': publisher.get('name', '') if isinstance(publisher, dict) else '',
        'Published': item.get('issued', ''),
        'Modified': item.get('modified', ''),
        'URL': item.get('landingPage', [''])[0] if item.get('landingPage') else ''
    })

# Save to CSV
df_all = pd.DataFrame(all_records)
df_all.to_csv('all_datasets_unfiltered.csv', index=False, encoding='utf-8-sig')

print(f"✓ Saved {len(df_all)} datasets to: all_datasets_unfiltered.csv")

=== Processing and saving unfiltered data ===
✓ Saved 10000 datasets to: all_datasets_unfiltered.csv


In [51]:
import pandas as pd

INCLUSION = ['study', 'analysis', 'economy', 'economic', 'research', 'statistics', 'survey', 'benchmarking']
EXCLUSION = ['construction', 'health care', 'transport', 'mobility', 'sport', 'culture', 'infrastructure']

print("=== Filtering datasets by keywords ===")

de_data, fr_data, it_data, en_data = [], [], [], []

for item in datasets_raw:
    title = item.get('title', {})
    desc = item.get('description', {})
    publisher = item.get('publisher', {})
    
    # Process each language
    for lang, lang_list in [('de', de_data), ('fr', fr_data), ('it', it_data), ('en', en_data)]:
        if lang in title or lang in desc:
            text = f"{title.get(lang, '')} {desc.get(lang, '')}".lower()
            
            # Apply filters
            if any(k.lower() in text for k in INCLUSION) and not any(k.lower() in text for k in EXCLUSION):
                lang_list.append({
                    'Title': title.get(lang, 'N/A'),
                    'Description': str(desc.get(lang, 'N/A'))[:500],
                    'Publisher': publisher.get('name', 'N/A') if isinstance(publisher, dict) else 'N/A',
                    'Published': item.get('issued', 'N/A'),
                    'Modified': item.get('modified', 'N/A'),
                    'URL': item.get('landingPage', ['N/A'])[0] if item.get('landingPage') else 'N/A'
                })

print(f"German: {len(de_data)} | French: {len(fr_data)} | Italian: {len(it_data)} | English: {len(en_data)}")

# Save to Excel with language sheets
if any([de_data, fr_data, it_data, en_data]):
    with pd.ExcelWriter('filtered_datasets_by_language.xlsx', engine='openpyxl') as writer:
        if de_data:
            pd.DataFrame(de_data).to_excel(writer, sheet_name='German', index=False)
        if fr_data:
            pd.DataFrame(fr_data).to_excel(writer, sheet_name='French', index=False)
        if it_data:
            pd.DataFrame(it_data).to_excel(writer, sheet_name='Italian', index=False)
        if en_data:
            pd.DataFrame(en_data).to_excel(writer, sheet_name='English', index=False)
    
    print(f"\n✓ Saved filtered datasets to: filtered_datasets_by_language.xlsx")
else:
    print("\n✗ No datasets matched the filters")

=== Filtering datasets by keywords ===
German: 8 | French: 3 | Italian: 40 | English: 258

✓ Saved filtered datasets to: filtered_datasets_by_language.xlsx


In [1]:
import requests
import pandas as pd
from tqdm import tqdm

# ---------- PART 1: Fetch main search results with filters ----------
url = "https://data.europa.eu/api/hub/search/search"
headers = {"accept": "application/json"}

# Date range filter (like stat_tm_1 and stat_tm_2 in SIMAP)
search_filters = {
    "minDate": "2000-01-01T00:00:00Z",
    "maxDate": "2025-12-31T23:59:59Z",
    "q": "economic market research benchmarking survey statistics analysis",
    "filters": "dataset",
    "limit": 100,
    "sort": "issued+desc"
}

all_results = []
page = 0

while page < 100:
    search_filters["page"] = page
    
    r = requests.get(url, headers=headers, params=search_filters, timeout=30)
    
    if r.status_code != 200:
        break
    
    data = r.json().get("result", {})
    results = data.get("results", [])
    
    if not results:
        break
    
    all_results.extend(results)
    print(f"Fetched page {page} ({len(results)} records)")
    page += 1

df_main = pd.DataFrame(all_results)
print(f"\n✅ Total fetched: {len(df_main)} records")

# ---------- PART 2: Keep only required fields ----------
required_fields = ["id", "title", "description", "issued", "modified", "publisher", "landingPage", "country"]
df_filtered = df_main[[col for col in required_fields if col in df_main.columns]].copy()

# ---------- PART 3: Filter out exclusions ----------
EXCLUSION = ['construction', 'transport', 'health', 'sport', 'infrastructure', 'INSPIRE', 'cadastre']

def should_keep(row):
    text = str(row.get('title', '')) + ' ' + str(row.get('description', ''))
    return not any(ex.lower() in text.lower() for ex in EXCLUSION)

df_filtered = df_filtered[df_filtered.apply(should_keep, axis=1)]
print(f"✅ After filtering: {len(df_filtered)} records")

# ---------- PART 4: Extract multi-language fields ----------
final_data = []

for _, item in tqdm(df_filtered.iterrows(), total=len(df_filtered), desc="Processing languages"):
    title = item.get('title', {}) if isinstance(item.get('title'), dict) else {}
    desc = item.get('description', {}) if isinstance(item.get('description'), dict) else {}
    publisher = item.get('publisher', {}) if isinstance(item.get('publisher'), dict) else {}
    
    final_data.append({
        'id': item.get('id', ''),
        'title_en': title.get('en', ''),
        'title_de': title.get('de', ''),
        'title_fr': title.get('fr', ''),
        'description_en': str(desc.get('en', ''))[:300],
        'description_de': str(desc.get('de', ''))[:300],
        'publisher': publisher.get('name', ''),
        'published': item.get('issued', ''),
        'modified': item.get('modified', ''),
        'url': item.get('landingPage', [''])[0] if isinstance(item.get('landingPage'), list) else ''
    })

df_final = pd.DataFrame(final_data)

# ---------- PART 5: Save ----------
output = "/Users/cedricjansen/bak-economics/notebooks/data_europa_datasets.csv"
df_final.to_csv(output, index=False, encoding='utf-8-sig')

print(f"\n💾 Saved as data_europa_datasets.csv")
print(f"✅ Final count: {len(df_final)} datasets")
print("\nSample:")
print(df_final[['title_en', 'publisher', 'published']].head(3))

Fetched page 0 (100 records)
Fetched page 1 (100 records)
Fetched page 2 (100 records)
Fetched page 3 (100 records)
Fetched page 4 (100 records)
Fetched page 5 (100 records)
Fetched page 6 (100 records)
Fetched page 7 (100 records)
Fetched page 8 (100 records)
Fetched page 9 (100 records)
Fetched page 10 (100 records)
Fetched page 11 (100 records)
Fetched page 12 (100 records)
Fetched page 13 (100 records)
Fetched page 14 (100 records)
Fetched page 15 (100 records)
Fetched page 16 (100 records)
Fetched page 17 (100 records)
Fetched page 18 (100 records)
Fetched page 19 (100 records)
Fetched page 20 (100 records)
Fetched page 21 (100 records)
Fetched page 22 (100 records)
Fetched page 23 (100 records)
Fetched page 24 (100 records)
Fetched page 25 (100 records)
Fetched page 26 (100 records)
Fetched page 27 (100 records)
Fetched page 28 (100 records)
Fetched page 29 (100 records)
Fetched page 30 (100 records)
Fetched page 31 (100 records)
Fetched page 32 (100 records)
Fetched page 33 (100

Processing languages: 100%|██████████| 9830/9830 [00:00<00:00, 48069.09it/s]


💾 Saved as data_europa_datasets.csv
✅ Final count: 9830 datasets

Sample:
                         title_en               publisher            published
0            Profile coefficients                  Enedis  2025-10-08T02:15:55
1     Parking spaces PMR – Angers  Angers Loire Métropole  2025-10-07T23:00:04
2  Bicycle parking racks – Angers  Angers Loire Métropole  2025-10-07T22:00:11


In [26]:
import requests
import pandas as pd

# Set up constants
API_URL = "https://ted.europa.eu/TED/content/api/rest/json"
headers = {"accept": "application/json"}

# Filters
min_date = "2018-01-01"
max_date = "2024-12-31"
classification_codes = [
    "72000000","79300000","73100000","79311400","72314000",
    "79416000","72320000","98300000","79310000","79000000","79311410"
]
# Join the list into a comma-separated string
classification_codes_str = ",".join(classification_codes)
keywords = "Study OR Analysis OR Economy OR Benchmarking OR Wirtschaftsberatung"

# Initialize list to store all tenders
all_tenders = []

# Fetch multiple pages
for page in range(1, 6):  # Adjust total pages as needed
    json_payload = {
  "query": "classification-cpv IN (72000000, 79300000, 73100000, 79311400, 72314000, 79416000, 72320000, 98300000, 79310000, 79000000, 79311410)",
  "fields": [
    "classification-cpv",
    "title-part",
    "publication-date"
  ],
  "page": page,
  "limit": 50,
  "scope": "ACTIVE",
  "paginationMode": "PAGE_NUMBER"
}



    # prepare params
    response = requests.get(API_URL, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        results = data.get('results', [])
        if not results:
            break  # <-- properly inside the loop
        all_tenders.extend(results)
        print(f"Page {page} fetched with {len(results)} notices.")
    else:
        print(f"Error at page {page}: {response.status_code}")
        break  # <-- also properly inside the loop


#Save data and filter out unwanted tenders

df.to_csv("tenders_raw_data.csv", index=False)


exclusion_keywords = [
    "construction", "bau", "health care", "gesundheit", "transport", "verkehr",
    "mobility", "mobilität", "sport", "culture", "kultur", "street", "strasse",
    "infrastructure", "infrastruktur", "process", "prozess", "IT"
]

def is_relevant(row):
    text = (row.get('title', '') + ' ' + row.get('description', '')).lower()
    return not any(ex_kw.lower() in text for ex_kw in exclusion_keywords)

filtered_df = df[df.apply(is_relevant, axis=1)]

filtered_df.to_csv ("tenders_filtered.csv", index=False)

print(f"Total after filtering: {len(filtered_df)}")

# Extract relevant fields and prepare for analysis

columns_of_interest = [
    "title", "publicationDate", "description", "url", "classification-cpv", "region", "buyer_name"
]


existing_columns = [col for col in columns_of_interest if col in filtered_df.columns]

final_df = filtered_df[existing_columns]

final_df.to_csv("curated_tenders.csv", index=False)


print(f"Curated data saved with {len(final_df)} records.")



JSONDecodeError: Expecting value: line 35 column 4 (char 40)

In [27]:
print("Status code:", response.status_code)
print("Response content (first 500 chars):")
print(response.text[:500])



Status code: 200
Response content (first 500 chars):
































	
		
			<!DOCTYPE html>




























































<html class="ltr" dir="ltr" lang="en-GB">

<head>
	<title>TED - EU Tenders, the Supplement to the Official Journal - TED</title>

	<meta content="initial-scale=1.0, width=device-width" name="viewport" />

	<script defer src="https://europa.eu/webtools/load.js" type="text/javascript"></script>























In [38]:
import requests
import math

# Your API URL
API_URL = "https://api.ted.europa.eu/v3/notices/search"

headers = {
    "accept": "*/*",
    "Content-Type": "application/json",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

# Define your date range and CPV codes
min_date = "2018-01-01"
max_date = "2024-12-31"

# Step 1: Get total count with limit=1
payload = {
  "query": "classification-cpv IN (72000000, 79300000, 73100000, 79311400, 72314000, 79416000, 72320000, 98300000, 79310000, 79000000, 79311410)",
  "fields": ["classification-cpv", "title-part", "publication-date"],
  "page": 1,
  "limit": 1,
  "scope": "ACTIVE",
  "paginationMode": "PAGE_NUMBER"
}

response = requests.post(API_URL, headers=headers, json=payload)
if response.status_code == 200:
    data = response.json()
    total_notices = data.get("totalNoticeCount", 0)
    print(f"Total notices in date range: {total_notices}")
else:
    print("Error fetching total count.")
    total_notices = 0

# Step 2: Calculate number of pages (limit=250)
max_limit = 250
total_pages = math.ceil(total_notices / max_limit)

# Step 3: Loop through pages, fetch notices
all_notices = []

for page in range(1, total_pages + 1):
    payload = {
      "query": "classification-cpv IN (72000000, 79300000, 73100000, 79311400, 72314000, 79416000, 72320000, 98300000, 79310000, 79000000, 79311410)",
      "fields": ["classification-cpv", "title-part", "publication-date"],
      "page": page,
      "limit": max_limit,
      "scope": "ACTIVE",
      "paginationMode": "PAGE_NUMBER"
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    if response.status_code == 200:
        notices = response.json().get("notices", [])
        if not notices:
            break
        all_notices.extend(notices)
        print(f"Page {page}: {len(notices)} notices")
    else:
        print(f"Error at page {page}. Status code: {response.status_code}")
        break

# After collecting, you can convert to DataFrame or save to CSV
import pandas as pd
df = pd.read_csv("all_notices.csv")
print(df.columns)

keywords = ["BFS", "Bundesamt für Statistik", "research", "economic"]


mask = df['title-part'].str.contains('|'.join(keywords), case=False, na=False)
filtered_df = df[mask]

filtered_df.to_excel("filtered_notices.xlsx", index=False)


print(f"Total notices in raw data: {len(df)}")
print(f"Notices after filtering: {len(filtered_notices)}")

Total notices in date range: 55931
Page 1: 250 notices
Page 2: 250 notices
Page 3: 250 notices
Page 4: 250 notices
Page 5: 250 notices
Page 6: 250 notices
Page 7: 250 notices
Page 8: 250 notices
Page 9: 250 notices
Page 10: 250 notices
Page 11: 250 notices
Page 12: 250 notices
Page 13: 250 notices
Page 14: 250 notices
Page 15: 250 notices
Page 16: 250 notices
Page 17: 250 notices
Page 18: 250 notices
Page 19: 250 notices
Page 20: 250 notices
Page 21: 250 notices
Page 22: 250 notices
Page 23: 250 notices
Page 24: 250 notices
Page 25: 250 notices
Page 26: 250 notices
Page 27: 250 notices
Page 28: 250 notices
Page 29: 250 notices
Page 30: 250 notices
Page 31: 250 notices
Page 32: 250 notices
Page 33: 250 notices
Page 34: 250 notices
Page 35: 250 notices
Page 36: 250 notices
Page 37: 250 notices
Page 38: 250 notices
Page 39: 250 notices
Page 40: 250 notices
Page 41: 250 notices
Page 42: 250 notices
Page 43: 250 notices
Page 44: 250 notices
Page 45: 250 notices
Page 46: 250 notices
Page 47:

NameError: name 'filtered_notices' is not defined

In [43]:
import requests
import math

# Your API URL
API_URL = "https://api.ted.europa.eu/v3/notices/search"

headers = {
    "accept": "*/*",
    "Content-Type": "application/json",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

# Define your date range and CPV codes
min_date = "2018-01-01"
max_date = "2024-12-31"

# Step 1: Get total count with limit=1
payload = {
  "query": "classification-cpv IN (72000000, 79300000, 73100000, 79311400, 72314000, 79416000, 72320000, 98300000, 79310000, 79000000, 79311410)",
  "fields": ["classification-cpv", "title-part", "publication-date"],
  "page": 1,
  "limit": 1,
  "scope": "ACTIVE",
  "paginationMode": "PAGE_NUMBER"
}

response = requests.post(API_URL, headers=headers, json=payload)
if response.status_code == 200:
    data = response.json()
    total_notices = data.get("totalNoticeCount", 0)
    print(f"Total notices in date range: {total_notices}")
else:
    print("Error fetching total count.")
    total_notices = 0

# Step 2: Calculate number of pages (limit=250)
max_limit = 250
total_pages = math.ceil(total_notices / max_limit)

# Step 3: Loop through pages, fetch notices
all_notices = []

for page in range(1, total_pages + 1):
    payload = {
      "query": "classification-cpv IN ('72000000')",
      "fields": ["classification-cpv", "title-part", "publication-date"],
      "page": page,
      "limit": max_limit,
      "scope": "ACTIVE",
      "paginationMode": "PAGE_NUMBER"
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    if response.status_code == 200:
        notices = response.json().get("notices", [])
        if not notices:
            break
        all_notices.extend(notices)
        print(f"Page {page}: {len(notices)} notices")
    else:
        print(f"Error at page {page}. Status code: {response.status_code}")
        break

# After collecting, you can convert to DataFrame or save to CSV
import pandas as pd
df = pd.read_csv("all_notices.csv")
print(df.columns)

keywords = ["BFS", "Bundesamt für Statistik", "research", "economic"]


mask = df['title-part'].str.contains('|'.join(keywords), case=False, na=False)
filtered_df = df[mask]

filtered_df.to_excel("filtered_notices.xlsx", index=False)


print(f"Total notices in raw data: {len(df)}")
print(f"Notices after filtering: {len(filtered_df)}")


Total notices in date range: 55931
Page 1: 250 notices
Page 2: 250 notices
Page 3: 250 notices
Page 4: 250 notices
Page 5: 250 notices
Page 6: 250 notices
Page 7: 250 notices
Page 8: 250 notices
Page 9: 250 notices
Page 10: 250 notices
Page 11: 250 notices
Page 12: 250 notices
Page 13: 250 notices
Page 14: 250 notices
Page 15: 250 notices
Page 16: 250 notices
Page 17: 250 notices
Page 18: 250 notices
Page 19: 250 notices
Page 20: 250 notices
Page 21: 250 notices
Page 22: 250 notices
Page 23: 250 notices
Page 24: 250 notices
Page 25: 250 notices
Page 26: 250 notices
Page 27: 250 notices
Page 28: 250 notices
Page 29: 250 notices
Page 30: 250 notices
Page 31: 250 notices
Page 32: 250 notices
Page 33: 250 notices
Page 34: 250 notices
Page 35: 250 notices
Page 36: 250 notices
Page 37: 250 notices
Page 38: 250 notices
Page 39: 250 notices
Page 40: 250 notices
Page 41: 250 notices
Page 42: 250 notices
Page 43: 250 notices
Page 44: 250 notices
Page 45: 250 notices
Page 46: 250 notices
Page 47:

In [101]:
import requests
import datetime
import time
import csv

# API endpoint
API_URL = "https://api.ted.europa.eu/v3/notices/search"

# Headers
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}

# Define a broad date range as needed (last 6 months here)
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=180)
start_str = start_date.strftime("%Y%m%d")
end_str = end_date.strftime("%Y%m%d")

# Build a generic query for recent notices (adjust as needed)
query = f"publication-date >= {start_str} AND publication-date <= {end_str}"

# Fields to retrieve; include description for full details if available
fields = ["classification-cpv", "title-part", "publication-date", "links", "description"]

# Set max notices to fetch
max_total_notices = 15000
limit_per_page = 250
page = 1
all_notices = []

while len(all_notices) < max_total_notices:
    payload = {
        "query": query,
        "fields": fields,
        "limit": limit_per_page,
        "scope": "ACTIVE",
        "paginationMode": "PAGE_NUMBER",
        "page": page
    }

    # Make the request
    response = requests.post(API_URL, headers=headers, json=payload)
    print(f"Requesting page {page}...")

    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        break

    data = response.json()
    notices = data.get("notices", [])
    print(f"Fetched {len(notices)} notices from page {page}.")

    # Add these notices to the total list
    all_notices.extend(notices)

    # Check if fewer than limit fetched -> last page
    if len(notices) < limit_per_page:
        print("No more notices or last page reached.")
        break

    # Check maximum limit
    if len(all_notices) >= max_total_notices:
        print(f"Reached the maximum of {max_total_notices} notices.")
        break

    # Delay to avoid rate limits
    time.sleep(1)
    page += 1

# Save all notices to CSV
filename = "notices_full.csv"
with open(filename, "w", encoding="utf-8", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["classification-cpv", "title-part", "publication-date", "url", "description"])

    for notice in all_notices:
        classification = ', '.join(notice.get("classification-cpv", []))
        url = notice.get("links", {}).get("html", {}).get("ENG", "")
        title = notice.get("title-part", "")
        pub_date = notice.get("publication-date", "")
        description = notice.get("description", "")
        writer.writerow([classification, title, pub_date, url, description])

print(f"Saved {len(all_notices)} notices to {filename}.")


Requesting page 1...
Error 400: {"message":"Parameter 'fields' contains unsupported value (supported values are: sme-part,touchpoint-gateway-ted-esen,submission-url-lot,organisation-person-addinfo-part,no-negocaition-necessary-lot,BT-13(t)-Part,organisation-city-serv-prov,result-framework-maximum-value-cur-notice,BT-821-Lot,touchpoint-partname-tenderer,touchpoint-internet-address-paying,BT-805-Lot,touchpoint-post-code-addinfo-lot,option-description-lot,touchpoint-internet-address-fiscal-legis-lot,organisation-email-tenderer,organisation-country-buyer,organisation-post-code-docprovider-part,tendering-party-name,BT-5101(a)-Lot,place-performance-streetline1-part,BT-137-LotsGroup,organisation-person-mediator-lot,BT-711-LotResult-Currency,organisation-gateway-revieworg-lot,OPP-130-Business,organisation-street-ted-esen,OPP-021-Contract,touchpoint-tel-subcontractor,ubo-email,touchpoint-internet-address-tender-receip-part,touchpoint-internet-address-tender-eval-part,BT-5131-Part,BT-1501(n)-Con

In [107]:
import requests
import datetime
import time
import csv

# API endpoint and headers
API_URL = "https://api.ted.europa.eu/v3/notices/search"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}

# Date range: last 6 months
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=180)

start_str = start_date.strftime("%Y%m%d")
end_str = end_date.strftime("%Y%m%d")

# Multiple CPV codes
cpv_codes = [
    "72000000", "79300000", "73100000", "79311400", "72314000",
    "79416000", "72320000", "98300000", "79310000", "79000000",
    "79311410"
]

# Build CPV query string
cpv_query = " OR ".join([f"classification-cpv IN ({code})" for code in cpv_codes])

# Full query with date range and CPV codes
query = f"publication-date >= {start_str} AND publication-date <= {end_str} AND ({cpv_query})"

# Fields supported by the API (excluding unsupported ones)
fields = ["classification-cpv", "title-part", "publication-date", "links"]

# Pagination parameters
limit = 250
page = 1
max_notices = 15000

all_notices = []

while len(all_notices) < max_notices:
    payload = {
        "query": query,
        "fields": fields,
        "limit": limit,
        "scope": "ACTIVE",
        "paginationMode": "PAGE_NUMBER",
        "page": page
    }

    response = requests.post(API_URL, headers=headers, json=payload)
    print(f"Requesting page {page}...")

    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        break

    data = response.json()
    notices = data.get("notices", [])
    print(f"Fetched {len(notices)} notices from page {page}.")

    all_notices.extend(notices)

    # Stop if fewer notices than limit (last page)
    if len(notices) < limit:
        print("No more notices or data exhausted.")
        break

    if len(all_notices) >= max_notices:
        print("Reached the maximum of 15,000 notices.")
        break

    time.sleep(1)  # optional delay to respect rate limits
    page += 1

# Save notices to CSV
csv_filename = "notices_full.csv"
with open(csv_filename, "w", encoding="utf-8", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["classification-cpv", "title", "publication-date", "url"])

    for notice in all_notices:
        class_cpv = ', '.join(notice.get("classification-cpv", []))
        title = notice.get("title-part", "")
        pub_date = notice.get("publication-date", "")
        url = notice.get("links", {}).get("html", {}).get("ENG", "")
        writer.writerow([class_cpv, title, pub_date, url])

print(f"All notices saved to '{csv_filename}'.")


Requesting page 1...
Fetched 250 notices from page 1.
Requesting page 2...
Fetched 250 notices from page 2.
Requesting page 3...
Fetched 250 notices from page 3.
Requesting page 4...
Fetched 250 notices from page 4.
Requesting page 5...
Fetched 250 notices from page 5.
Requesting page 6...
Fetched 250 notices from page 6.
Requesting page 7...
Fetched 250 notices from page 7.
Requesting page 8...
Fetched 250 notices from page 8.
Requesting page 9...
Fetched 250 notices from page 9.
Requesting page 10...
Fetched 250 notices from page 10.
Requesting page 11...
Fetched 250 notices from page 11.
Requesting page 12...
Fetched 250 notices from page 12.
Requesting page 13...
Fetched 250 notices from page 13.
Requesting page 14...
Fetched 250 notices from page 14.
Requesting page 15...
Fetched 250 notices from page 15.
Requesting page 16...
Fetched 250 notices from page 16.
Requesting page 17...
Fetched 250 notices from page 17.
Requesting page 18...
Fetched 250 notices from page 18.
Requesting

In [112]:
import requests
import datetime
import time
import pandas as pd
from langdetect import detect

# --- 1. Fetch notices (up to 15,000) ---
API_URL = "https://api.ted.europa.eu/v3/notices/search"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}

# Date range for last 6 months
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=180)

start_str = start_date.strftime("%Y%m%d")
end_str = end_date.strftime("%Y%m%d")

# Your CPV codes
cpv_codes = [
    "72000000", "79300000", "73100000", "79311400", "72314000",
    "79416000", "72320000", "98300000", "79310000", "79000000",
    "79311410"
]
# Build CPV query
cpv_query = " OR ".join([f"classification-cpv IN ({code})" for code in cpv_codes])

# Main query
query = f"publication-date >= {start_str} AND publication-date <= {end_str} AND ({cpv_query})"
fields = ["classification-cpv", "title-part", "publication-date", "links"]
limit = 250
page = 1
max_notices = 15000

all_notices = []

# Loop to get up to 15,000 notices
while len(all_notices) < max_notices:
    payload = {
        "query": query,
        "fields": fields,
        "limit": limit,
        "scope": "ACTIVE",
        "paginationMode": "PAGE_NUMBER",
        "page": page
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    print(f"Requesting page {page}...")
    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        break
    data = response.json()
    notices = data.get("notices", [])
    print(f"Fetched {len(notices)} notices from page {page}.")
    all_notices.extend(notices)
    if len(notices) < limit:
        print("No more notices or last page reached.")
        break
    if len(all_notices) >= max_notices:
        print("Reached the 15,000 notices limit.")
        break
    time.sleep(1)
    page += 1

print(f"\nTotal notices fetched: {len(all_notices)}")


# --- 2. Detect language for each notice and prepare data ---
lang_map = {'de': 'German', 'en': 'English', 'fr': 'French', 'it': 'Italian'}
notices_with_lang = []

for notice in all_notices:
    text_for_detection = str(notice.get("title-part", ""))
    try:
        lang_code = detect(text_for_detection)
        language = lang_map.get(lang_code, 'unknown')
    except:
        language = 'unknown'
    
    notices_with_lang.append({
        "classification-cpv": ', '.join(notice.get("classification-cpv", [])),
        "title": notice.get("title-part", ""),
        "publication-date": notice.get("publication-date", ""),
        "url": notice.get("links", {}).get("html", {}).get("ENG", ""),
        "language": language
    })

# --- 3. Save all notices to an Excel with sheets for each language ---
df = pd.DataFrame(notices_with_lang)

with pd.ExcelWriter("notices_by_language.xlsx") as writer:
    for lang in df["language"].unique():
        df[df["language"] == lang].to_excel(writer, sheet_name=lang, index=False)

# --- 4. Save all notices to CSV ---
df.to_csv("all_notices_detailed.csv", index=False)

print("All notices saved to 'notices_by_language.xlsx' and 'all_notices_detailed.csv'.")



ModuleNotFoundError: No module named 'langdetect'

In [41]:
print(df['title-part'].head(10))


0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
Name: title-part, dtype: object


In [89]:
import os
import re
import pandas as pd
import pdfplumber

# Define paths and file name
base_dir = os.path.join(os.path.expanduser("~"), "bak-economics", "data", "processsed")
os.makedirs(base_dir, exist_ok=True)
excel_path = os.path.join(base_dir, "2025-10-09_tenders_body.xlsx")
main_folder_path = '/Users/cedricjansen/bak-economics/data/processsed/tenders_pdfs'  # Correct path

# Utility functions
clean = lambda text: re.sub(r'.*(disclaimer|legal).*', '', text, flags=re.IGNORECASE).strip()

def extract_date_content(text):
    date_match = re.search(r'\b(\d{1,2}[./-]\d{1,2}[./-]\d{2,4})\b', text)
    return date_match.group(1) if date_match else None

def extract_date_filename(filename):
    date_match = re.search(r'(\d{4}-\d{2}-\d{2})', filename)
    if date_match:
        return date_match.group(1)
    date_match = re.search(r'(\d{8})', filename)
    if date_match:
        year = date_match.group(1)[:4]
        month = date_match.group(1)[4:6]
        day = date_match.group(1)[6:]
        return f"{day}.{month}.{year}"
    return None

# Data processing
data = []
for year_folder in os.listdir(main_folder_path):
    year_path = os.path.join(main_folder_path, year_folder)
    if os.path.isdir(year_path) and year_folder.isdigit() and len(year_folder) == 4:
        for filename in os.listdir(year_path):
            if filename.endswith('.pdf'):
                file_path = os.path.join(year_path, filename)
                try:
                    with pdfplumber.open(file_path) as pdf:
                        full_text = '\n'.join(page.extract_text() for page in pdf.pages if page.extract_text())
                except:
                    print(f"Error reading {filename}")
                    continue
                # Title Extraction
                try:
                    title = full_text.split('\n')[0] if full_text else 'No Title'
                except Exception as e:
                    title = f'Error: No Title - {filename}'
                    print(f"Error extracting title from {filename}: {e}")
                # Body Extraction
                try:
                    body = full_text
                except Exception as e:
                    body = f'Error: No Body - {filename}'
                    print(f"Error extracting body from {filename}: {e}")
                pub_date = extract_date_content(full_text) or extract_date_filename(filename) or year_folder
                data.append({'Title': title, 'Publication Date': pub_date, 'Body': clean(body)})

# Export to Excel
pd.DataFrame(data).to_excel(excel_path, index=False)




Cannot set gray non-stroke color because /'P124' is an invalid float value
Cannot set gray non-stroke color because /'P126' is an invalid float value


In [79]:
#TED API Notice Search v3 - CORRECT WORKING VERSION

#API Endpoint: POST https://api.ted.europa.eu/v3/notices/search

VERIFIED WORKING REQUEST STRUCTURE:
{
  "query": "classification-cpv IN (72000000)",
  "fields": ["classification-cpv", "title-part", "publication-date"],
  "page": 1,
  "limit": 10,
  "scope": "ACTIVE",
  "paginationMode": "PAGE_NUMBER"
}

Response Structure:
{
  "notices": [
    {
      "classification-cpv": ["72224000", "76000000"],
      "publication-number": "146763-2016",
      "publication-date": "2016-04-28+02:00",
      "title-part": "...",
      "links": {...}
    }
  ]
}
"""

import requests
import json
from typing import Dict, List, Optional


class TEDSearchAPI:
    
#Client for TED API notice-search-v3 with classification filtering.

    BASE_URL = "https://api.ted.europa.eu/v3/notices/search"

    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({
            "Content-Type": "application/json",
            "Accept": "application/json"
        })

    def search(
        self,
        query: str,
        fields: List[str],
        page: int = 1,
        limit: int = 10,
        scope: str = "ACTIVE",
        pagination_mode: str = "PAGE_NUMBER"
    ) -> Dict:
        """
        #Execute a search query against TED API.

        Args:
            query: Expert query string (e.g., "classification-cpv IN (72000000)")
            fields: List of fields to return in results
            page: Page number (default: 1)
            limit: Results per page (default: 10, max: 100)
            scope: Search scope - "ACTIVE" or "ALL" (default: ACTIVE)
            pagination_mode: "PAGE_NUMBER" or "ITERATION_TOKEN"

        Returns:
            Dict with "notices" array containing search results
        """
        payload = {
            "query": query,
            "fields": fields,
            "page": page,
            "limit": limit,
            "scope": scope,
            "paginationMode": pagination_mode
        }

        print(f"🔍 Querying TED API...")
        print(f"Query: {query}")
        print(f"Page: {page}, Limit: {limit}\n")

        try:
            response = self.session.post(self.BASE_URL, json=payload)
            response.raise_for_status()
            result = response.json()

            notices = result.get('notices', [])
            print(f"✅ Found {len(notices)} results on page {page}\n")

            return result
        except requests.exceptions.HTTPError as e:
            print(f"❌ HTTP Error: {e}")
            if hasattr(e.response, 'text'):
                error_data = json.loads(e.response.text)
                print(f"Error Details:\n{json.dumps(error_data, indent=2)}")
            raise
        except Exception as e:
            print(f"❌ Error: {e}")
            raise

    def search_by_cpv(
        self,
        cpv_codes: List[str],
        exclude_cpv: Optional[List[str]] = None,
        fields: Optional[List[str]] = None,
        page: int = 1,
        limit: int = 10
    ) -> Dict:
        """
        Search notices by CPV (Common Procurement Vocabulary) codes.

        Args:
            cpv_codes: List of CPV codes to include (e.g., ["72000000"])
            exclude_cpv: Optional list of CPV codes to exclude (e.g., ["721*"])
            fields: Fields to return (default: classification, title, date)
            page: Page number
            limit: Results per page

        Returns:
            Search results with "notices" array

        Common CPV Codes:
            71000000 - Architectural, construction, engineering services
            72000000 - IT services (computer programming, consultancy)
            73000000 - Research and experimental development services
            75000000 - Administration, defence and social security services
            80000000 - Education and training services
            90000000 - Sewage, refuse, cleaning services
        """
        # Build query
        query_parts = []

        if cpv_codes:
            codes_str = " ".join(cpv_codes)
            query_parts.append(f"classification-cpv IN ({codes_str})")

        if exclude_cpv:
            exclude_str = " ".join(exclude_cpv)
            query_parts.append(f"classification-cpv NOT IN ({exclude_str})")

        query = " AND ".join(query_parts)

        # Default fields
        if not fields:
            fields = ["classification-cpv", "title-part", "publication-date"]

        return self.search(query, fields, page=page, limit=limit)

    def print_request_details(
        self,
        query: str,
        fields: List[str],
        page: int = 1,
        limit: int = 10
    ):
        """Print the full API request details for debugging."""
        payload = {
            "query": query,
            "fields": fields,
            "page": page,
            "limit": limit,
            "scope": "ACTIVE",
            "paginationMode": "PAGE_NUMBER"
        }

        print("="*70)
        print("FULL API CALL DETAILS")
        print("="*70)
        print(f"\nHTTP Method: POST")
        print(f"URL: {self.BASE_URL}")
        print(f"\nHeaders:")
        print(f"  Content-Type: application/json")
        print(f"  Accept: application/json")
        print(f"\nRequest Body:")
        print(json.dumps(payload, indent=2))
        print(f"\n{'='*70}\n")


def display_results(results: Dict, max_results: int = 5):
    """Pretty print search results."""
    notices = results.get('notices', [])

    print(f"{'='*70}")
    print(f"RESULTS: Showing {min(len(notices), max_results)} of {len(notices)} notices")
    print(f"{'='*70}\n")

    for i, notice in enumerate(notices[:max_results], 1):
        print(f"📄 Notice #{i}")
        print(f"   Publication: {notice.get('publication-number', 'N/A')}")
        print(f"   Date: {notice.get('publication-date', 'N/A')}")
        print(f"   CPV Codes: {', '.join(notice.get('classification-cpv', []))}")

        title = notice.get('title-part', 'N/A')
        if isinstance(title, str) and len(title) > 100:
            title = title[:100] + "..."
        print(f"   Title: {title}")

        # Show link to HTML version
        html_links = notice.get('links', {}).get('html', {})
        if 'ENG' in html_links:
            print(f"   URL: {html_links['ENG']}")

        print()


def main():
    

    client = TEDSearchAPI()

    print("\n" + "="*70)
    print("TED API CLASSIFICATION FILTER - WORKING EXAMPLES")
    print("="*70 + "\n")

    # Example 1: Simple CPV search
    print("\n📋 EXAMPLE 1: Search by CPV Code (IT Services - 72000000)")
    print("-" * 70 + "\n")

    try:
        results = client.search_by_cpv(
            cpv_codes=["72000000"],
            limit=5
        )
        display_results(results)
    except Exception as e:
        print(f"❌ Failed: {e}\n")

    # Example 2: Print request details
    print("\n📋 EXAMPLE 2: Print Full API Request")
    print("-" * 70 + "\n")

    client.print_request_details(
        query="classification-cpv IN (72000000)",
        fields=["classification-cpv", "title-part", "publication-date"],
        page=1,
        limit=10
    )

    # Example 3: Multiple CPV codes
    print("\n📋 EXAMPLE 3: Multiple CPV Codes (IT + R&D)")
    print("-" * 70 + "\n")

    try:
        results = client.search_by_cpv(
            cpv_codes=["72000000", "73000000"],
            fields=["classification-cpv", "title-part", "publication-date"],
            limit=5
        )
        display_results(results)
    except Exception as e:
        print(f"❌ Failed: {e}\n")

    # Example 4: CPV with exclusions
    print("\n📋 EXAMPLE 4: CPV with Exclusions")
    print("-" * 70 + "\n")

    try:
        results = client.search_by_cpv(
            cpv_codes=["72000000"],
            exclude_cpv=["722*"],  # Exclude specific subcategories
            limit=5
        )
        display_results(results)
    except Exception as e:
        print(f"❌ Failed: {e}\n")

    # Example 5: Pagination
    print("\n📋 EXAMPLE 5: Pagination (Page 2)")
    print("-" * 70 + "\n")

    try:
        results = client.search_by_cpv(
            cpv_codes=["72000000"],
            page=2,
            limit=3
        )
        display_results(results)
    except Exception as e:
        print(f"❌ Failed: {e}\n")

    print("\n" + "="*70)
    print("✅ All examples completed!")
    print("="*70 + "\n")


if __name__ == "__main__":
    main()


SyntaxError: unterminated triple-quoted string literal (detected at line 288) (4023925665.py, line 208)

In [99]:
import requests
import json
import datetime
import time

# API endpoint and headers
API_URL = "https://api.ted.europa.eu/v3/notices/search"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}

# Example: last 30 days, format as YYYYMMDD
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=30)

start_str = start_date.strftime("%Y%m%d")
end_str = end_date.strftime("%Y%m%d")

# Query with correct date format
query = f"publication-date >= {start_str} AND publication-date <= {end_str} AND classification-cpv IN (72000000)"

# Fields
fields = ["classification-cpv", "title-part", "publication-date", "links"]

# Pagination
limit = 250
page = 1

all_notices = []

while True:
    payload = {
        "query": query,
        "fields": fields,
        "limit": limit,
        "scope": "ACTIVE",
        "paginationMode": "PAGE_NUMBER",
        "page": page
    }

    response = requests.post(API_URL, headers=headers, json=payload)
    print(f"Requesting page {page}...")

    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        break

    data = response.json()
    notices = data.get("notices", [])
    print(f"Fetched {len(notices)} notices from page {page}.")

    # Add the current page notices
    all_notices.extend(notices)

    if len(notices) < limit:
        print("No more notices or reached the end.")
        break

    time.sleep(1)
    page += 1

print(f"\nTotal notices collected: {len(all_notices)}")
# Show first 5 notices
for i, notice in enumerate(all_notices[:5], 1):
    print(f"\nNotice {i}:")
    print(json.dumps(notice, indent=2))


Requesting page 1...
Fetched 250 notices from page 1.
Requesting page 2...
Fetched 250 notices from page 2.
Requesting page 3...
Fetched 250 notices from page 3.
Requesting page 4...
Fetched 250 notices from page 4.
Requesting page 5...
Fetched 250 notices from page 5.
Requesting page 6...
Fetched 250 notices from page 6.
Requesting page 7...
Fetched 250 notices from page 7.
Requesting page 8...
Fetched 250 notices from page 8.
Requesting page 9...
Fetched 250 notices from page 9.
Requesting page 10...
Fetched 250 notices from page 10.
Requesting page 11...
Fetched 250 notices from page 11.
Requesting page 12...
Fetched 250 notices from page 12.
Requesting page 13...
Fetched 250 notices from page 13.
Requesting page 14...
Fetched 250 notices from page 14.
Requesting page 15...
Fetched 250 notices from page 15.
Requesting page 16...
Fetched 250 notices from page 16.
Requesting page 17...
Fetched 250 notices from page 17.
Requesting page 18...
Fetched 250 notices from page 18.
Requesting

In [114]:
import requests
import datetime
import time
import pandas as pd
from langdetect import detect

# --- 1. Fetch notices (up to 15,000) ---
API_URL = "https://api.ted.europa.eu/v3/notices/search"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}

# Date range for last 6 months
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=180)

start_str = start_date.strftime("%Y%m%d")
end_str = end_date.strftime("%Y%m%d")

# Your CPV codes
cpv_codes = [
    "72000000", "79300000", "73100000", "79311400", "72314000",
    "79416000", "72320000", "98300000", "79310000", "79000000",
    "79311410"
]
# Build CPV query
cpv_query = " OR ".join([f"classification-cpv IN ({code})" for code in cpv_codes])

# Main query
query = f"publication-date >= {start_str} AND publication-date <= {end_str} AND ({cpv_query})"
fields = ["classification-cpv", "title-part", "publication-date", "links"]
limit = 250
page = 1
max_notices = 15000

all_notices = []

# Loop to get up to 15,000 notices
while len(all_notices) < max_notices:
    payload = {
        "query": query,
        "fields": fields,
        "limit": limit,
        "scope": "ACTIVE",
        "paginationMode": "PAGE_NUMBER",
        "page": page
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    print(f"Requesting page {page}...")
    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        break
    data = response.json()
    notices = data.get("notices", [])
    print(f"Fetched {len(notices)} notices from page {page}.")
    all_notices.extend(notices)
    if len(notices) < limit:
        print("No more notices or last page reached.")
        break
    if len(all_notices) >= max_notices:
        print("Reached the 15,000 notices limit.")
        break
    time.sleep(1)
    page += 1

print(f"\nTotal notices fetched: {len(all_notices)}")


# --- 2. Detect language for each notice and prepare data ---
lang_map = {'de': 'German', 'en': 'English', 'fr': 'French', 'it': 'Italian'}
notices_with_lang = []

for notice in all_notices:
    text_for_detection = str(notice.get("title-part", ""))
import requests
import datetime
import time
import pandas as pd
from langdetect import detect

# --- Fetch notices (up to 15,000) ---
API_URL = "https://api.ted.europa.eu/v3/notices/search"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}

end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=180)

start_str = start_date.strftime("%Y%m%d")
end_str = end_date.strftime("%Y%m%d")

# CPV codes
cpv_codes = [
    "72000000", "79300000", "73100000", "79311400", "72314000",
    "79416000", "72320000", "98300000", "79310000", "79000000",
    "79311410"
]
cpv_query = " OR ".join([f"classification-cpv IN ({code})" for code in cpv_codes])
query = f"publication-date >= {start_str} AND publication-date <= {end_str} AND ({cpv_query})"
fields = ["classification-cpv", "title-part", "publication-date", "links"]
limit = 250
page = 1
max_notices = 15000

all_notices = []

while len(all_notices) < max_notices:
    payload = {
        "query": query,
        "fields": fields,
        "limit": limit,
        "scope": "ACTIVE",
        "paginationMode": "PAGE_NUMBER",
        "page": page
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        break
    data = response.json()
    notices = data.get("notices", [])
    all_notices.extend(notices)
    if len(notices) < limit or len(all_notices) >= max_notices:
        break
    time.sleep(1)
    page += 1

# --- Detect language and filter by keywords ---
lang_map = {'de': 'German', 'en': 'English', 'fr': 'French', 'it': 'Italian'}
relevant_keywords = [
    "study", "studie", "analysis", "analyse", "economy", "wirtschaft", "ökonomie",
    "benchmarking", "wirtschaftsberatung", "haupt-cpv", "beratung", "dienstleistung",
    "ausschreibung", "offenes verfahren", "bks", "bundesamt", "seco", "kanton",
    "index", "regionen", "wirtschaftsforschung"
]
exclude_keywords = [
    "construction", "bau", "health care", "gesundheit", "transport", "verkehr",
    "mobility", "mobilität", "sport", "culture", "kultur", "street", "strasse",
    "infrastructure", "infrastruktur", "process", "prozess", "it"
]

filtered_notices = []

for notice in all_notices:
    title = str(notice.get("title-part", ""))
    try:
        lang_code = detect(title)
        language = lang_map.get(lang_code, 'unknown')
    except:
        language = 'unknown'

    combined_text = title.lower()
    
    if any(kw.lower() in combined_text for kw in relevant_keywords) and not any(kw.lower() in combined_text for kw in exclude_keywords):
        filtered_notices.append({
            "classification-cpv": ', '.join(notice.get("classification-cpv", [])),
            "title": notice.get("title-part", ""),
            "publication-date": notice.get("publication-date", ""),
            "url": notice.get("links", {}).get("html", {}).get("ENG", ""),
            "language": language
        })

# --- Save filtered notices to Excel with language sheets ---
df_filtered = pd.DataFrame(filtered_notices)

with pd.ExcelWriter("filtered_relevant_notices.xlsx") as writer:
    for lang in df_filtered["language"].unique():
        df_filtered[df_filtered["language"] == lang].to_excel(writer, sheet_name=lang, index=False)

# Save to CSV
df_filtered.to_csv("relevant_notices.csv", index=False)

print("Filtered relevant notices saved to 'filtered_relevant_notices.xlsx' and 'relevant_notices.csv'.")



Requesting page 1...
Fetched 250 notices from page 1.
Requesting page 2...
Fetched 250 notices from page 2.
Requesting page 3...
Fetched 250 notices from page 3.
Requesting page 4...
Fetched 250 notices from page 4.
Requesting page 5...
Fetched 250 notices from page 5.
Requesting page 6...
Fetched 250 notices from page 6.
Requesting page 7...
Fetched 250 notices from page 7.
Requesting page 8...
Fetched 250 notices from page 8.
Requesting page 9...
Fetched 250 notices from page 9.
Requesting page 10...
Fetched 250 notices from page 10.
Requesting page 11...
Fetched 250 notices from page 11.
Requesting page 12...
Fetched 250 notices from page 12.
Requesting page 13...
Fetched 250 notices from page 13.
Requesting page 14...
Fetched 250 notices from page 14.
Requesting page 15...
Fetched 250 notices from page 15.
Requesting page 16...
Fetched 250 notices from page 16.
Requesting page 17...
Fetched 250 notices from page 17.
Requesting page 18...
Fetched 250 notices from page 18.
Requesting

In [128]:
import requests
import datetime
import time
import pandas as pd
from langdetect import detect

# API URL
API_URL = "https://api.ted.europa.eu/v3/notices/search"

# Headers
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}

# Date range: last 6 months
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=180)
start_str = start_date.strftime("%Y%m%d")
end_str = end_date.strftime("%Y%m%d")

# CPV codes
cpv_codes = [
    "72000000", "79300000", "73100000", "79311400", "72314000",
    "79416000", "72320000", "98300000", "79310000", "79000000",
    "79311410"
]
cpv_query = " OR ".join([f"classification-cpv IN ({code})" for code in cpv_codes])

# Query
query = f"publication-date >= {start_str} AND publication-date <= {end_str} AND ({cpv_query})"

# Fields (no 'description', which is unsupported)
fields = ["classification-cpv", "title-part", "publication-date", "links"]

# Pagination setup
limit = 250
page = 1
max_notices = 15000

all_notices = []

# Fetch notices with pagination
while len(all_notices) < max_notices:
    payload = {
        "query": query,
        "fields": fields,
        "limit": limit,
        "scope": "ACTIVE",
        "paginationMode": "PAGE_NUMBER",
        "page": page
    }

    response = requests.post(API_URL, headers=headers, json=payload)
    print(f"Requesting page {page}...")

    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        break

    data = response.json()
    notices = data.get("notices", [])
    print(f"Fetched {len(notices)} notices from page {page}.")

    all_notices.extend(notices)

    if len(notices) < limit:
        print("No more notices or last page reached.")
        break
    if len(all_notices) >= max_notices:
        print("Reached the 15,000 notices limit.")
        break

    time.sleep(1)
    page += 1

# --- Save all notices to CSV ---
csv_filename = "notices_full.csv"
with open(csv_filename, "w", encoding="utf-8", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["classification-cpv", "title", "publication-date", "url"])
    for notice in all_notices:
        cpv_str = ', '.join(notice.get("classification-cpv", []))
        title = notice.get("title-part", "")
        pub_date = notice.get("publication-date", "")
        url = notice.get("links", {}).get("html", {}).get("ENG", "")
        writer.writerow([cpv_str, title, pub_date, url])

print(f"All notices saved to '{csv_filename}'.")



Requesting page 1...
Fetched 250 notices from page 1.
Requesting page 2...
Fetched 250 notices from page 2.
Requesting page 3...
Fetched 250 notices from page 3.
Requesting page 4...
Fetched 250 notices from page 4.
Requesting page 5...
Fetched 250 notices from page 5.
Requesting page 6...
Fetched 250 notices from page 6.
Requesting page 7...
Fetched 250 notices from page 7.
Requesting page 8...
Fetched 250 notices from page 8.
Requesting page 9...
Fetched 250 notices from page 9.
Requesting page 10...
Fetched 250 notices from page 10.
Requesting page 11...
Fetched 250 notices from page 11.
Requesting page 12...
Fetched 250 notices from page 12.
Requesting page 13...
Fetched 250 notices from page 13.
Requesting page 14...
Fetched 250 notices from page 14.
Requesting page 15...
Fetched 250 notices from page 15.
Requesting page 16...
Fetched 250 notices from page 16.
Requesting page 17...
Fetched 250 notices from page 17.
Requesting page 18...
Fetched 250 notices from page 18.
Requesting

In [75]:
import os

# Define the paths
base_dir = '/Users/cedricjansen/bak-economics/data/processed'
tenders_pdfs_dir = os.path.join(base_dir, 'tenders_pdfs')
year_dir = os.path.join(tenders_pdfs_dir, '2020')  # Example year directory

# Check if directories exist
print(f"Checking if base directory '{base_dir}' exists: {os.path.exists(base_dir)}")
if os.path.exists(base_dir):
    print(f"Checking if '{base_dir}' is a directory: {os.path.isdir(base_dir)}")

print(f"Checking if tenders_pdfs directory '{tenders_pdfs_dir}' exists: {os.path.exists(tenders_pdfs_dir)}")
if os.path.exists(tenders_pdfs_dir):
    print(f"Checking if '{tenders_pdfs_dir}' is a directory: {os.path.isdir(tenders_pdfs_dir)}")

print(f"Checking if year directory '{year_dir}' exists: {os.path.exists(year_dir)}")
if os.path.exists(year_dir):
    print(f"Checking if '{year_dir}' is a directory: {os.path.isdir(year_dir)}")


Checking if base directory '/Users/cedricjansen/bak-economics/data/processed' exists: True
Checking if '/Users/cedricjansen/bak-economics/data/processed' is a directory: True
Checking if tenders_pdfs directory '/Users/cedricjansen/bak-economics/data/processed/tenders_pdfs' exists: False
Checking if year directory '/Users/cedricjansen/bak-economics/data/processed/tenders_pdfs/2020' exists: False


In [131]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from langdetect import detect
import time

# Step 1: Function to get all notice URLs from the main listings page
def get_notice_urls():
    base_url = "https://ted.europa.eu/en/notice"
    response = requests.get(base_url)
    if response.status_code != 200:
        print(f"Failed to load main page: {response.status_code}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    links = []

    # Find all relevant links (inspect the actual class or pattern)
    for a in soup.find_all('a', href=True):
        href = a['href']
        if "/en/notice/" in href:
            if href.startswith('/'):
                links.append("https://ted.europa.eu" + href)
            else:
                links.append(href)

    print(f"Found {len(links)} notices.")
    return links

# Step 2: Function to scrape individual notice details
def scrape_notice(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to load {url}")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')

    # Update these selectors after inspecting actual webpage
    try:
        full_text = soup.find('div', class_='notice__description').get_text(separator=' ', strip=True)
    except:
        full_text = 'N/A'

    try:
        org_name = soup.find('div', class_='notice__organization').get_text(strip=True)
    except:
        org_name = 'N/A'

    try:
        title = soup.find('h1', class_='notice__title').get_text(strip=True)
    except:
        title = 'N/A'

    # Extract CPV code
    cpv = 'N/A'
    match = re.search(r'\d{8}', full_text)
    if match:
        cpv = match.group()

    # Publication date
    pub_date = 'N/A'
    pub_div = soup.find('div', class_='notice__publication-date')
    if pub_div:
        pub_date = pub_div.get_text(strip=True)

    # Deadlines (example: from the info on the page)
    deadline_submit = 'N/A'
    deadline_additional = 'N/A'
    for span in soup.find_all('span'):
        txt = span.get_text().lower()
        date_match = re.search(r'\d{1,2}\.\d{1,2}\.\d{4}', txt)
        if date_match:
            date_str = date_match.group()
            if 'submit' in txt:
                deadline_submit = date_str
            elif 'additional' in txt or 'info' in txt:
                deadline_additional = date_str

    # Determine language
    try:
        lang_code = detect(full_text)
        lang_map = {'de': 'German', 'en': 'English', 'fr': 'French', 'it': 'Italian'}
        language = lang_map.get(lang_code, 'unknown')
    except:
        language = 'unknown'

    return {
        'Full Text': full_text,
        'Organization': org_name,
        'Title': title,
        'CPV': cpv,
        'Publication Date': pub_date,
        'Deadline for Submission': deadline_submit,
        'Deadline for Additional Info': deadline_additional,
        'URL': url,
        'Language': language
    }

# Step 3: Main script
def main():
    notice_urls = get_notice_urls()
    notices = []

    for i, url in enumerate(notice_urls):
        print(f"Processing {i+1}/{len(notice_urls)}: {url}")
        data = scrape_notice(url)
        if data:
            notices.append(data)
        time.sleep(1)  # be polite

    df = pd.DataFrame(notices)
    if df.empty:
        print("No data scraped. Exiting.")
        return

    # Save full data to CSV
    df.to_csv("ted_notices_full.csv", index=False)

    # Save


In [23]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from langdetect import detect
import time

# ---------------------------------------------------
# 1. Fetch notices using TED API (max 15k notices)
# ---------------------------------------------------
API_URL = "https://api.ted.europa.eu/v3/notices/search"
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}

start_date = "20180101"  # Jan 1, 2018
end_date = time.strftime("%Y%m%d")  # today

cpv_codes = [
    "72000000", "79300000", "73100000", "79311400", "72314000"
]
cpv_query = " OR ".join([f"classification-cpv IN ({code})" for code in cpv_codes])

query = f"publication-date >= {start_date} AND publication-date <= {end_date} AND ({cpv_query})"
fields = ["classification-cpv", "title-part", "publication-date", "links"]
limit = 250
page = 1
max_notices = 15000

notices_list = []

while len(notices_list) < max_notices:
    payload = {
        "query": query,
        "fields": fields,
        "limit": limit,
        "scope": "ACTIVE",
        "paginationMode": "PAGE_NUMBER",
        "page": page
    }
    resp = requests.post(API_URL, headers=headers, json=payload)
    if resp.status_code != 200:
        print(f"Error {resp.status_code}: {resp.text}")
        break
    data = resp.json()
    notices_page = data.get("notices", [])
    print(f"Page {page}: {len(notices_page)} notices")
    notices_list.extend(notices_page)
    if len(notices_page) < limit:
        break
    time.sleep(1)
    page += 1

print(f"Total notices fetched: {len(notices_list)}")

# ---------------------------------------------------
# 2. Visit each notice URL to extract full details
# ---------------------------------------------------
def get_notice_full_details(url):
    try:
        r = requests.get(url)
        if r.status_code != 200:
            print(f"Failed to load {url}")
            return None
        soup = BeautifulSoup(r.content, 'html.parser')

        # 2.1. Procedure - title
        proc_title_div = soup.find('h2', string=re.compile('Procedure', re.I))
        procedure_title = ''
        if proc_title_div:
            procedure_title = proc_title_div.get_text(strip=True)

        # 2.1. Procedure - description
        procedure_desc_div = proc_title_div.find_next_sibling('div') if proc_title_div else None
        procedure_description = ''
        if procedure_desc_div:
            procedure_description = procedure_desc_div.get_text(separator=' ', strip=True)

        # Full Text
        desc_div = soup.find('div', class_='notice__description')
        full_text = desc_div.get_text(separator=' ', strip=True) if desc_div else ''

        # Organization Name
        org_div = soup.find('div', class_='notice__organization')
        organization = org_div.get_text(strip=True) if org_div else 'N/A'

        # Title
        title_tag = soup.find('h1', class_='notice__title')
        title = title_tag.get_text(strip=True) if title_tag else 'N/A'

        # CPV
        cpv_match = re.search(r'\d{8}', full_text)
        cpv = cpv_match.group() if cpv_match else 'N/A'

        # Publication Date
        pub_div = soup.find('div', class_='notice__publication-date')
        publication_date = pub_div.get_text(strip=True) if pub_div else 'N/A'

        # Deadlines
        deadline_submit = 'N/A'
        deadline_additional = 'N/A'
        for span in soup.find_all('span'):
            txt = span.get_text().lower()
            date_match = re.search(r'\d{1,2}\.\d{1,2}\.\d{4}', txt)
            if date_match:
                date_str = date_match.group()
                if 'submit' in txt:
                    deadline_submit = date_str
                elif 'additional' in txt or 'info' in txt:
                    deadline_additional = date_str

        # Procurement value
        procurement_value = 'N/A'
        price_div = soup.find('div', class_='notice__value')  # update class accordingly
        if price_div:
            p = re.search(r'EUR\s*([\d,]+)', price_div.get_text())
            if p:
                procurement_value = f"EUR {p.group(1)}"

        try:
            if full_text.strip():
                lang_code = detect(full_text)
                lang_map = {'de':'German', 'en':'English', 'fr':'French', 'it':'Italian'}
                language = lang_map.get(lang_code, 'unknown')
            else:
                language = 'unknown'
        except:
            language = 'unknown'

        return {
            'Full Text': full_text,
            'Organization': organization,
            'Title': title,
            'CPV': cpv,
            'Publication Date': pub_date,
            'Deadline for Submission': deadline_submit,
            'Deadline for Additional Info': deadline_additional,
            'Procurement Value': procurement_value,
            'Language': language,
            'URL': url
        }
    except:
        return None



# During your loop over notice URLs
df = pd.DataFrame(notices_data)

# Define the columns to include
columns = [
    'Title',
    'CPV',
    'Full Text',                   # full content, can be empty
    'Organization',
    'Publication Date',
    'Deadline for Submission',
    'Deadline for Additional Info',
    'Procurement Value',
    'Language',
    'URL'
]

# Save to CSV
df.to_csv('ted_notices_full_details.csv', columns=columns, index=False, encoding='utf-8')

print("CSV saved as 'ted_notices_full_details.csv'")

Page 1: 250 notices
Page 2: 250 notices
Page 3: 250 notices
Page 4: 250 notices
Page 5: 250 notices
Page 6: 250 notices
Page 7: 250 notices
Page 8: 250 notices
Page 9: 250 notices
Page 10: 250 notices
Page 11: 250 notices
Page 12: 250 notices
Page 13: 250 notices
Page 14: 250 notices
Page 15: 250 notices
Page 16: 250 notices
Page 17: 250 notices
Page 18: 250 notices
Page 19: 250 notices
Page 20: 250 notices
Page 21: 250 notices
Page 22: 250 notices
Page 23: 250 notices
Page 24: 250 notices
Page 25: 250 notices
Page 26: 250 notices
Page 27: 250 notices
Page 28: 250 notices
Page 29: 250 notices
Page 30: 250 notices
Page 31: 250 notices
Page 32: 250 notices
Page 33: 250 notices
Page 34: 250 notices
Page 35: 250 notices
Page 36: 250 notices
Page 37: 250 notices
Page 38: 250 notices
Page 39: 250 notices
Page 40: 250 notices
Page 41: 250 notices
Page 42: 250 notices
Page 43: 250 notices
Page 44: 250 notices
Page 45: 250 notices
Page 46: 250 notices
Page 47: 250 notices
Page 48: 250 notices
P

KeyError: "None of [Index(['Title', 'CPV', 'Full Text', 'Organization', 'Publication Date',\n       'Deadline for Submission', 'Deadline for Additional Info',\n       'Procurement Value', 'Language', 'URL'],\n      dtype='object')] are in the [columns]"

In [96]:
import requests
import json
from typing import Dict, List, Optional

class TEDSearchAPI:
    """Client for TED API notice-search-v3 with classification filtering."""

    BASE_URL = "https://api.ted.europa.eu/v3/notices/search"

    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({
            "Content-Type": "application/json",
            "Accept": "application/json"
        })

    def search(
        self,
        query: str,
        fields: List[str],
        page: int = 1,
        limit: int = 10,
        scope: str = "ACTIVE",
        pagination_mode: str = "PAGE_NUMBER"
    ) -> Dict:
        """Execute a search query against TED API."""
        payload = {
            "query": query,
            "fields": fields,
            "page": page,
            "limit": limit,
            "scope": scope,
            "paginationMode": pagination_mode
        }

        print(f"🔍 Querying TED API...")
        print(f"Query: {query}")
        print(f"Page: {page}, Limit: {limit}\n")
        print("Payload being sent:")
        print(json.dumps(payload, indent=2))
        print("-" * 70)

        try:
            response = self.session.post(self.BASE_URL, json=payload)
            response.raise_for_status()
            result = response.json()
            notices = result.get('notices', [])
            print(f"✅ Found {len(notices)} results on page {page}\n")
            return result
        except requests.exceptions.HTTPError as e:
            print(f"❌ HTTP Error: {e}")
            if hasattr(e.response, 'text'):
                error_data = json.loads(e.response.text)
                print(f"Error Details:\n{json.dumps(error_data, indent=2)}")
            raise
        except Exception as e:
            print(f"❌ Error: {e}")
            raise

    def search_by_cpv(
        self,
        cpv_codes: List[str],
        exclude_cpv: Optional[List[str]] = None,
        fields: Optional[List[str]] = None,
        page: int = 1,
        limit: int = 10
    ) -> Dict:
        """Builds a query with CPV codes and executes search."""
        query_parts = []

        # Ensure cpv_codes are valid full-length codes
        if cpv_codes:
            # Join with spaces, e.g. '72000000' '73000000'
            codes_str = " ".join(cpv_codes)
            query_parts.append(f"classification-cpv IN ({codes_str})")

        if exclude_cpv:
            exclude_str = " ".join(exclude_cpv)
            query_parts.append(f"classification-cpv NOT IN ({exclude_str})")

        query = " AND ".join(query_parts)
        print(f"Constructed query: {query}")

        if not fields:
            fields = ["classification-cpv", "title-part", "publication-date"]

        return self.search(query, fields, page=page, limit=limit)

    def print_request_details(
        self,
        query: str,
        fields: List[str],
        page: int = 1,
        limit: int = 10
    ):
        """Print the full API request details for debugging."""
        payload = {
            "query": query,
            "fields": fields,
            "page": page,
            "limit": limit,
            "scope": "ACTIVE",
            "paginationMode": "PAGE_NUMBER"
        }

        print("="*70)
        print("FULL API CALL DETAILS")
        print("="*70)
        print(f"\nHTTP Method: POST")
        print(f"URL: {self.BASE_URL}")
        print(f"\nHeaders:")
        print(f"  Content-Type: application/json")
        print(f"  Accept: application/json")
        print(f"\nRequest Body:")
        print(json.dumps(payload, indent=2))
        print(f"\n{'='*70}\n")


def display_results(results: Dict, max_results: int = 5):
    """Pretty print search results."""
    notices = results.get('notices', [])
    print(f"{'='*70}")
    print(f"RESULTS: Showing {min(len(notices), max_results)} of {len(notices)} notices")
    print(f"{'='*70}\n")
    if not notices:
        print("No notices found.")
        return
    for i, notice in enumerate(notices[:max_results], 1):
        print(f"📄 Notice #{i}")
        print(f"   Publication: {notice.get('publication-number', 'N/A


SyntaxError: unterminated string literal (detected at line 130) (1753882307.py, line 130)

In [91]:
import json
import datetime
import requests
API_URL = "https://api.ted.europa.eu/v3/notices/search"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}

query = "publication-date >= today(-7) AND publication-date <= today() AND classification-cpv IN (72000000)"
json_payload = {
    "query": query,
    "fields": [
        "classification-cpv",
        "title-part",
        "publication-date",
        "links"
    ],
    "limit": 250,
    "scope": "ACTIVE",
    "paginationMode": "ITERATION"
}
response = requests.post(API_URL, headers=headers, json=json_payload)
print("Status code:", response.status_code)

if response.status_code == 200:
    data = response.json()
    print("Total notices fetched:", len(data.get("notices", [])))
    print("Response snippet:\n")
    import json
    print(json.dumps(data, indent=2))
else:
    print("Error:", response.text)

Status code: 200
Total notices fetched: 250
Response snippet:

{
  "notices": [
    {
      "classification-cpv": [
        "72000000",
        "72000000"
      ],
      "publication-number": "646953-2025",
      "publication-date": "2025-10-03+02:00",
      "links": {
        "xml": {
          "MUL": "https://ted.europa.eu/en/notice/646953-2025/xml"
        },
        "pdf": {
          "BUL": "https://ted.europa.eu/bg/notice/646953-2025/pdf",
          "SPA": "https://ted.europa.eu/es/notice/646953-2025/pdf",
          "CES": "https://ted.europa.eu/cs/notice/646953-2025/pdf",
          "DAN": "https://ted.europa.eu/da/notice/646953-2025/pdf",
          "DEU": "https://ted.europa.eu/de/notice/646953-2025/pdf",
          "EST": "https://ted.europa.eu/et/notice/646953-2025/pdf",
          "ELL": "https://ted.europa.eu/el/notice/646953-2025/pdf",
          "ENG": "https://ted.europa.eu/en/notice/646953-2025/pdf",
          "FRA": "https://ted.europa.eu/fr/notice/646953-2025/pdf",
      